# Question Formulation 
You should refine the question formulation based on the general task description you’ve been given, ie, predicting flight delays. This should include some discussion of why this is an important task from a business perspective, who the stakeholders are, etc.. Some literature review will be helpful to figure out how this problem is being solved now, and the State Of The Art (SOTA) in this domain. Introduce the goal of your analysis. What questions will you seek to answer, why do people perform this kind of analysis on this kind of data? Preview what level of performance your model would need to achieve to be practically useful.

## Problem
The problem in front of us goes beyond just predicting flight delays. Each flight contains passengers that are trying to get to their next flight or to their final destination. A delay of one flight can have cascading delays to other flights. Planes are far too expensive to just be discarded after they are used and extremely likely to be used again within the hour. This means there is also an optimization problem to solve on how to efficiently re-route planes or make use of spare resources (planes and crews) to ensure that the next flight leaves on time.

The FAA defines a flight delay as a flight that is 15 minutes or more off schedule. This means that a flight can be late for either departure or arrive and still qualify as a late flight. We'll also need to predict the delay two hours before the flight would take off. This means that the weather data will be from 2 hours before the flight was intended to take off.

In [3]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/"))

path name size dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/_SUCCESS _SUCCESS 0 dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/part-00000-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet part-00000-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet 1030519 dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/part-00001-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet part-00001-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet 1161448 dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/part-00003-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet part-00003-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet 1109451 dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/part-00015-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet part-00015-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet 815185 dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/part-00020-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet part-00020-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet 586922 dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/part-00023-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet part-00023-e7b7f524-7be7-4ec3-afce-c31684af3aca-c000.snappy.parquet 455595

In [4]:
display(dbutils.fs.ls("dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/"))

path name size dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/_SUCCESS _SUCCESS 0 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/_committed_6351095181992560034 _committed_6351095181992560034 6084 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/_started_6351095181992560034 _started_6351095181992560034 0 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00000-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3617-1-c000.snappy.parquet part-00000-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3617-1-c000.snappy.parquet 109256889 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00001-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3626-1-c000.snappy.parquet part-00001-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3626-1-c000.snappy.parquet 98375438 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00002-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3635-1-c000.snappy.parquet part-00002-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3635-1-c000.snappy.parquet 97627735 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00003-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3644-1-c000.snappy.parquet part-00003-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3644-1-c000.snappy.parquet 89677996 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00004-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3651-1-c000.snappy.parquet part-00004-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3651-1-c000.snappy.parquet 85150252 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00005-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3652-1-c000.snappy.parquet part-00005-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3652-1-c000.snappy.parquet 58641349 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00006-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3615-1-c000.snappy.parquet part-00006-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3615-1-c000.snappy.parquet 105917556 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00007-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3624-1-c000.snappy.parquet part-00007-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3624-1-c000.snappy.parquet 99676419 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00008-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3633-1-c000.snappy.parquet part-00008-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3633-1-c000.snappy.parquet 93829273 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00009-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3642-1-c000.snappy.parquet part-00009-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3642-1-c000.snappy.parquet 86008249 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00010-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3653-1-c000.snappy.parquet part-00010-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3653-1-c000.snappy.parquet 82240354 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00011-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3654-1-c000.snappy.parquet part-00011-tid-6351095181992560034-563bd7eb-71c5-45a8-839a-ed4a1f941eaf-3654-1-c000.snappy.parquet 60657771 dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/part-00012-tid-6351095181992560034-563bd

In [5]:
# Libraries used
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, NullType, ShortType, DateType, BooleanType, BinaryType
from pyspark.sql import SQLContext
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from pyspark.sql.functions import lit
import datetime as datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import sklearn.tree as tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate

In [6]:
##Start up Spark and SQL contexts
sc = spark.sparkContext
sqlContext = SQLContext(sc)
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.52.254.54:7077 
 AppName 
 Databricks Shell

# EDA & Discussion of Challenges
Determine 2-3 relevant EDA tasks that will help you make decisions about how you implement the algorithm to be scalable. Discuss any challenges that you anticipate based on the EDA you perform.

We have two datasets that we will leverage. One containing flight information and another with weather information. Let's start with breaking down the flight data. Our first task will be to create a binary column that contains whether the flight was delayed or not. 1 will represent that it delayed for either departure or arrival or both. 0 means that there is no delay.

There 109 total columns available in the flight data. Each provides information about the flight, but a number of these datum overlap with one another. To improve on the speed and efficiency of the data, we will need to do some dimension reduction to help eliminate highly correlated or useless columns. The dataset begins with 6 columns containing information relevant to the date of the flight. These are year, quarter, month, day_of_month, day_of_week, and fl_date.

## Explore Airlines Datasets

In [10]:
airlines = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/parquet_airlines_data_3m/*.parquet")
display(airlines.sample(False, 0.0001, seed=4))

YEAR QUARTER MONTH DAY_OF_MONTH DAY_OF_WEEK FL_DATE OP_UNIQUE_CARRIER OP_CARRIER_AIRLINE_ID OP_CARRIER TAIL_NUM OP_CARRIER_FL_NUM ORIGIN_AIRPORT_ID ORIGIN_AIRPORT_SEQ_ID ORIGIN_CITY_MARKET_ID ORIGIN ORIGIN_CITY_NAME ORIGIN_STATE_ABR ORIGIN_STATE_FIPS ORIGIN_STATE_NM ORIGIN_WAC DEST_AIRPORT_ID DEST_AIRPORT_SEQ_ID DEST_CITY_MARKET_ID DEST DEST_CITY_NAME DEST_STATE_ABR DEST_STATE_FIPS DEST_STATE_NM DEST_WAC CRS_DEP_TIME DEP_TIME DEP_DELAY DEP_DELAY_NEW DEP_DEL15 DEP_DELAY_GROUP DEP_TIME_BLK TAXI_OUT WHEELS_OFF WHEELS_ON TAXI_IN CRS_ARR_TIME ARR_TIME ARR_DELAY ARR_DELAY_NEW ARR_DEL15 ARR_DELAY_GROUP ARR_TIME_BLK CANCELLED CANCELLATION_CODE DIVERTED CRS_ELAPSED_TIME ACTUAL_ELAPSED_TIME AIR_TIME FLIGHTS DISTANCE DISTANCE_GROUP CARRIER_DELAY WEATHER_DELAY NAS_DELAY SECURITY_DELAY LATE_AIRCRAFT_DELAY FIRST_DEP_TIME TOTAL_ADD_GTIME LONGEST_ADD_GTIME DIV_AIRPORT_LANDINGS DIV_REACHED_DEST DIV_ACTUAL_ELAPSED_TIME DIV_ARR_DELAY DIV_DISTANCE DIV1_AIRPORT DIV1_AIRPORT_ID DIV1_AIRPORT_SEQ_ID DIV1_WHEELS_ON DIV1_TOTAL_GTIME DIV1_LONGEST_GTIME DIV1_WHEELS_OFF DIV1_TAIL_NUM DIV2_AIRPORT DIV2_AIRPORT_ID DIV2_AIRPORT_SEQ_ID DIV2_WHEELS_ON DIV2_TOTAL_GTIME DIV2_LONGEST_GTIME DIV2_WHEELS_OFF DIV2_TAIL_NUM DIV3_AIRPORT DIV3_AIRPORT_ID DIV3_AIRPORT_SEQ_ID DIV3_WHEELS_ON DIV3_TOTAL_GTIME DIV3_LONGEST_GTIME DIV3_WHEELS_OFF DIV3_TAIL_NUM DIV4_AIRPORT DIV4_AIRPORT_ID DIV4_AIRPORT_SEQ_ID DIV4_WHEELS_ON DIV4_TOTAL_GTIME DIV4_LONGEST_GTIME DIV4_WHEELS_OFF DIV4_TAIL_NUM DIV5_AIRPORT DIV5_AIRPORT_ID DIV5_AIRPORT_SEQ_ID DIV5_WHEELS_ON DIV5_TOTAL_GTIME DIV5_LONGEST_GTIME DIV5_WHEELS_OFF DIV5_TAIL_NUM 2015 1 1 15 4 2015-01-15 AA 19805 AA N496AA 1473 10397 1039705 30397 ATL Atlanta, GA GA 13 Georgia 34 11298 1129803 30194 DFW Dallas/Fort Worth, TX TX 48 Texas 74 1343 1349 6.0 6.0 0.0 0 1300-1359 28.0 1417 1520 6.0 1515 1526 11.0 11.0 0.0 0 1500-1559 0.0 null 0.0 152.0 157.0 123.0 1.0 731.0 3 null null null null null null null null 0 null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null 2015 1 1 24 6 2015-01-24 DL 19790 DL N988DL 2632 10397 1039705 30397 ATL Atlanta, GA GA 13 Georgia 34 10599 1059904 30599 BHM Birmingham, AL AL 1 Alabama 51 2153 2153 0.0 0.0 0.0 0 2100-2159 16.0 2209 2134 4.0 2148 2138 -10.0 0.0 0.0 -1 2100-2159 0.0 null 0.0 55.0 45.0 25.0 1.0 134.0 1 null null null null null null null null 0 null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null 2015 1 1 24 6 2015-01-24 UA 19977 UA N69838 1530 13930 1393003 30977 ORD Chicago, IL IL 17 Illinois 41 11618 1161802 31703 EWR Newark, NJ NJ 34 New Jersey 21 1311 1329 18.0 18.0 1.0 1 1300-1359 30.0 1359 1638 6.0 1622 1644 22.0 22.0 1.0 1 1600-1659 0.0 null 0.0 131.0 135.0 99.0 1.0 719.0 3 18.0 0.0 4.0 0.0 0.0 null null null 0 null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null 2015 1 1 29 4 2015-01-29 EV 20366 EV N14972 3920 13930 1393003 30977 ORD Chicago, IL IL 17 Illinois 41 10434 1043403 30434 AVP Scranton/Wilkes-Barre, PA PA 42 Pennsylvania 23 1325 1319 -6.0 0.0 0.0 -1 1300-1359 24.0 1343 1607 3.0 1610 1610 0.0 0.0 0.0 0 1600-1659 0.0 null 0.0 105.0 111.0 84.0 1.0 632.0 3 null null null null null null null null 0 null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null null 2015 1 1 31 6 2015-01-31 MQ 20398 MQ N633MQ 2990 13930 1393003 30977 ORD Chicago, IL IL 17 Illinois 41 12945 1294503 32945 LEX Lexington, KY KY 21 Kentucky 52 910 903 -7.0 0.0 0.0 -1 09

In [11]:
## This will show the structure of the airline dataset
airlines.printSchema()

root
-- YEAR: integer (nullable = true)
-- QUARTER: integer (nullable = true)
-- MONTH: integer (nullable = true)
-- DAY_OF_MONTH: integer (nullable = true)
-- DAY_OF_WEEK: integer (nullable = true)
-- FL_DATE: string (nullable = true)
-- OP_UNIQUE_CARRIER: string (nullable = true)
-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
-- OP_CARRIER: string (nullable = true)
-- TAIL_NUM: string (nullable = true)
-- OP_CARRIER_FL_NUM: integer (nullable = true)
-- ORIGIN_AIRPORT_ID: integer (nullable = true)
-- ORIGIN_AIRPORT_SEQ_ID: integer (nullable = true)
-- ORIGIN_CITY_MARKET_ID: integer (nullable = true)
-- ORIGIN: string (nullable = true)
-- ORIGIN_CITY_NAME: string (nullable = true)
-- ORIGIN_STATE_ABR: string (nullable = true)
-- ORIGIN_STATE_FIPS: integer (nullable = true)
-- ORIGIN_STATE_NM: string (nullable = true)
-- ORIGIN_WAC: integer (nullable = true)
-- DEST_AIRPORT_ID: integer (nullable = true)
-- DEST_AIRPORT_SEQ_ID: integer (nullable = true)
-- DEST_CITY_MARKET_ID: integer (nullable = true)
-- DEST: string (nullable = true)
-- DEST_CITY_NAME: string (nullable = true)
-- DEST_STATE_ABR: string (nullable = true)
-- DEST_STATE_FIPS: integer (nullable = true)
-- DEST_STATE_NM: string (nullable = true)
-- DEST_WAC: integer (nullable = true)
-- CRS_DEP_TIME: integer (nullable = true)
-- DEP_TIME: integer (nullable = true)
-- DEP_DELAY: double (nullable = true)
-- DEP_DELAY_NEW: double (nullable = true)
-- DEP_DEL15: double (nullable = true)
-- DEP_DELAY_GROUP: integer (nullable = true)
-- DEP_TIME_BLK: string (nullable = true)
-- TAXI_OUT: double (nullable = true)
-- WHEELS_OFF: integer (nullable = true)
-- WHEELS_ON: integer (nullable = true)
-- TAXI_IN: double (nullable = true)
-- CRS_ARR_TIME: integer (nullable = true)
-- ARR_TIME: integer (nullable = true)
-- ARR_DELAY: double (nullable = true)
-- ARR_DELAY_NEW: double (nullable = true)
-- ARR_DEL15: double (nullable = true)
-- ARR_DELAY_GROUP: integer (nullable = true)
-- ARR_TIME_BLK: string (nullable = true)
-- CANCELLED: double (nullable = true)
-- CANCELLATION_CODE: string (nullable = true)
-- DIVERTED: double (nullable = true)
-- CRS_ELAPSED_TIME: double (nullable = true)
-- ACTUAL_ELAPSED_TIME: double (nullable = true)
-- AIR_TIME: double (nullable = true)
-- FLIGHTS: double (nullable = true)
-- DISTANCE: double (nullable = true)
-- DISTANCE_GROUP: integer (nullable = true)
-- CARRIER_DELAY: double (nullable = true)
-- WEATHER_DELAY: double (nullable = true)
-- NAS_DELAY: double (nullable = true)
-- SECURITY_DELAY: double (nullable = true)
-- LATE_AIRCRAFT_DELAY: double (nullable = true)
-- FIRST_DEP_TIME: integer (nullable = true)
-- TOTAL_ADD_GTIME: double (nullable = true)
-- LONGEST_ADD_GTIME: double (nullable = true)
-- DIV_AIRPORT_LANDINGS: integer (nullable = true)
-- DIV_REACHED_DEST: double (nullable = true)
-- DIV_ACTUAL_ELAPSED_TIME: double (nullable = true)
-- DIV_ARR_DELAY: double (nullable = true)
-- DIV_DISTANCE: double (nullable = true)
-- DIV1_AIRPORT: string (nullable = true)
-- DIV1_AIRPORT_ID: integer (nullable = true)
-- DIV1_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV1_WHEELS_ON: integer (nullable = true)
-- DIV1_TOTAL_GTIME: double (nullable = true)
-- DIV1_LONGEST_GTIME: double (nullable = true)
-- DIV1_WHEELS_OFF: integer (nullable = true)
-- DIV1_TAIL_NUM: string (nullable = true)
-- DIV2_AIRPORT: string (nullable = true)
-- DIV2_AIRPORT_ID: integer (nullable = true)
-- DIV2_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV2_WHEELS_ON: integer (nullable = true)
-- DIV2_TOTAL_GTIME: double (nullable = true)
-- DIV2_LONGEST_GTIME: double (nullable = true)
-- DIV2_WHEELS_OFF: integer (nullable = true)
-- DIV2_TAIL_NUM: string (nullable = true)
-- DIV3_AIRPORT: string (nullable = true)
-- DIV3_AIRPORT_ID: integer (nullable = true)
-- DIV3_AIRPORT_SEQ_ID: integer (nullable = true)
-- DIV3_WHEELS_ON: integer (nullable = true)
-- DIV3_TOTAL_GTIME: double (nullable = true)
-- DIV3_LONGEST_GTIME: double (nullable = true)
-- DIV3_WHEELS_

In [12]:
df_airlines = airlines.toPandas()

In [13]:
df_airlines.describe()

YEAR 
 QUARTER 
 MONTH 
 DAY_OF_MONTH 
 DAY_OF_WEEK 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN_STATE_FIPS 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST_STATE_FIPS 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ARR_DELAY_GROUP 
 CANCELLED 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 DISTANCE_GROUP 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 
 
 
 
 count 
 161057.0 
 161057.0 
 161057.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 1.610570e+05 
 161057.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 1.610570e+05 
 161057.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 156231.000000 
 156231.000000 
 156231.000000 
 156231.000000 
 156231.000000 
 156121.000000 
 156121.000000 
 155957.000000 
 155957.000000 
 161057.000000 
 155957.000000 
 155692.000000 
 155692.000000 
 155692.000000 
 155692.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 155692.000000 
 155692.000000 
 161057.0 
 161057.000000 
 161057.000000 
 34915.000000 
 34915.000000 
 34915.00000 
 34915.000000 
 34915.000000 
 1053.000000 
 1053.000000 
 1053.000000 
 161057.000000 
 381.000000 
 265.000000 
 265.000000 
 381.000000 
 429.000000 
 4.290000e+02 
 429.000000 
 429.000000 
 429.000000 
 278.000000 
 17.000000 
 1.700000e+01 
 17.000000 
 17.000000 
 17.000000 
 5.000000 
 1.0 
 1.0 
 1.0 
 1.0 
 1.0 
 
 
 mean 
 2015.0 
 1.0 
 2.036552 
 15.591505 
 3.916520 
 20010.918153 
 2508.798711 
 11963.802592 
 1.196384e+06 
 30654.216389 
 14.773906 
 37.104336 
 12707.705111 
 1.270773e+06 
 32018.143148 
 28.618719 
 49.025475 
 1422.252768 
 1437.216897 
 12.583828 
 14.368870 
 0.234813 
 0.252427 
 18.062086 
 1465.591503 
 1563.578172 
 6.149086 
 1584.805361 
 1566.326064 
 7.511568 
 14.163483 
 0.224257 
 0.008729 
 0.030946 
 0.002366 
 124.434784 
 120.037144 
 95.836177 
 1.0 
 662.231340 
 3.124546 
 19.903050 
 6.769039 
 12.16838 
 0.017557 
 19.076672 
 1399.992403 
 36.226021 
 35.970560 
 0.005116 
 0.695538 
 347.101887 
 232.633962 
 47.908136 
 12499.242424 
 1.249927e+06 
 1464.046620 
 27.405594 
 22.242424 
 1474.884892 
 12674.176471 
 1.267421e+06 
 1341.411765 
 11.941176 
 10.235294 
 1392.200000 
 10397.0 
 1039705.0 
 51.0 
 4.0 
 4.0 
 
 
 std 
 0.0 
 0.0 
 0.832660 
 8.679528 
 1.995815 
 314.626497 
 1746.419264 
 1755.181564 
 1.755172e+05 
 288.141893 
 1.987185 
 3.477575 
 1522.429280 
 1.522428e+05 
 1388.348035 
 16.113962 
 21.488664 
 453.970474 
 463.857461 
 36.393693 
 35.596949 
 0.423883 
 2.142223 
 9.087123 
 465.871793 
 488.380807 
 4.818137 
 467.098093 
 494.175085 
 39.550690 
 36.196183 
 0.417093 
 2.305651 
 0.173171 
 0.048580 
 64.537094 
 63.288804 
 61.754708 
 0.0 
 474.807024 
 1.839999 
 38.098095 
 29.870353 
 25.07071 
 0.805233 
 36.269235 
 482.825093 
 28.256333 
 27.885904 
 0.163356 
 0.460784 
 204.471624 
 206.620475 
 124.824801 
 1657.100266 
 1.657094e+05 
 564.358918 
 25.880631 
 24.029032 
 543.504354 
 1805.210349 
 1.805201e+05 
 617.786073 
 9.222734 
 7.198345 
 586.312374 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 min 
 2015.0 
 1.0 
 1.000000 
 1.000000 
 1.000000 
 19393.000000 
 3.000000 
 10

In [14]:
# Unique value counts of each column
for col in df_airlines.columns:
  print(col, df_airlines[col].nunique())
  
# From DIV4 and DIV5 related columns do not have values, we could remove them

YEAR 1
QUARTER 1
MONTH 3
DAY_OF_MONTH 31
DAY_OF_WEEK 7
FL_DATE 90
OP_UNIQUE_CARRIER 13
OP_CARRIER_AIRLINE_ID 13
OP_CARRIER 13
TAIL_NUM 3907
OP_CARRIER_FL_NUM 3853
ORIGIN_AIRPORT_ID 2
ORIGIN_AIRPORT_SEQ_ID 2
ORIGIN_CITY_MARKET_ID 2
ORIGIN 2
ORIGIN_CITY_NAME 2
ORIGIN_STATE_ABR 2
ORIGIN_STATE_FIPS 2
ORIGIN_STATE_NM 2
ORIGIN_WAC 2
DEST_AIRPORT_ID 192
DEST_AIRPORT_SEQ_ID 192
DEST_CITY_MARKET_ID 175
DEST 192
DEST_CITY_NAME 188
DEST_STATE_ABR 51
DEST_STATE_FIPS 51
DEST_STATE_NM 51
DEST_WAC 51
CRS_DEP_TIME 978
DEP_TIME 1244
DEP_DELAY 473
DEP_DELAY_NEW 451
DEP_DEL15 2
DEP_DELAY_GROUP 15
DEP_TIME_BLK 19
TAXI_OUT 137
WHEELS_OFF 1259
WHEELS_ON 1279
TAXI_IN 111
CRS_ARR_TIME 1055
ARR_TIME 1281
ARR_DELAY 524
ARR_DELAY_NEW 461
ARR_DEL15 2
ARR_DELAY_GROUP 15
ARR_TIME_BLK 19
CANCELLED 2
CANCELLATION_CODE 3
DIVERTED 2
CRS_ELAPSED_TIME 316
ACTUAL_ELAPSED_TIME 480
AIR_TIME 450
FLIGHTS 1
DISTANCE 291
DISTANCE_GROUP 10
CARRIER_DELAY 315
WEATHER_DELAY 286
NAS_DELAY 250
SECURITY_DELAY 21
LATE_AIRCRAFT_DELAY 300
FIRST_DEP_TIME 650
TOTAL_ADD_GTIME 129
LONGEST_ADD_GTIME 126
DIV_AIRPORT_LANDINGS 4
DIV_REACHED_DEST 2
DIV_ACTUAL_ELAPSED_TIME 199
DIV_ARR_DELAY 181
DIV_DISTANCE 69
DIV1_AIRPORT 98
DIV1_AIRPORT_ID 98
DIV1_AIRPORT_SEQ_ID 98
DIV1_WHEELS_ON 348
DIV1_TOTAL_GTIME 93
DIV1_LONGEST_GTIME 80
DIV1_WHEELS_OFF 243
DIV1_TAIL_NUM 250
DIV2_AIRPORT 7
DIV2_AIRPORT_ID 7
DIV2_AIRPORT_SEQ_ID 7
DIV2_WHEELS_ON 17
DIV2_TOTAL_GTIME 10
DIV2_LONGEST_GTIME 10
DIV2_WHEELS_OFF 5
DIV2_TAIL_NUM 5
DIV3_AIRPORT 1
DIV3_AIRPORT_ID 1
DIV3_AIRPORT_SEQ_ID 1
DIV3_WHEELS_ON 1
DIV3_TOTAL_GTIME 1
DIV3_LONGEST_GTIME 1
DIV3_WHEELS_OFF 0
DIV3_TAIL_NUM 0
DIV4_AIRPORT 0
DIV4_AIRPORT_ID 0
DIV4_AIRPORT_SEQ_ID 0
DIV4_WHEELS_ON 0
DIV4_TOTAL_GTIME 0
DIV4_LONGEST_GTIME 0
DIV4_WHEELS_OFF 0
DIV4_TAIL_NUM 0
DIV5_AIRPORT 0
DIV5_AIRPORT_ID 0
DIV5_AIRPORT_SEQ_ID 0
DIV5_WHEELS_ON 0
DIV5_TOTAL_GTIME 0
DIV5_LONGEST_GTIME 0
DIV5_WHEELS_OFF 0
DIV5_TAIL_NUM 0

## Split the  Airlines dataset
- airlines_eda (5%)
- airlines_other (95%)

In [16]:
#repartition ensures that there is no possibility of overlap between data splits. See: https://medium.com/udemy-engineering/pyspark-under-the-hood-randomsplit-and-sample-inconsistencies-examined-7c6ec62644bc
airlines_eda, airlines_other = airlines.repartition(20,'FL_DATE').randomSplit([0.05,0.95], seed=4)
airlines_eda = airlines_eda.toPandas()
airlines_other = airlines_other.toPandas()

In [17]:
print("Row counts of airlines_eda: ", airlines_eda.count()[0])
print("Row counts of airlines_other: ", airlines_other.count()[0])

Row counts of airlines_eda: 8125
Row counts of airlines_other: 152932

## Explore airlines_eda Datasets

In [19]:
airlines_eda.head()

YEAR 
 QUARTER 
 MONTH 
 DAY_OF_MONTH 
 DAY_OF_WEEK 
 FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN 
 ORIGIN_CITY_NAME 
 ORIGIN_STATE_ABR 
 ORIGIN_STATE_FIPS 
 ORIGIN_STATE_NM 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST 
 DEST_CITY_NAME 
 DEST_STATE_ABR 
 DEST_STATE_FIPS 
 DEST_STATE_NM 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 ... 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 
 
 
 
 0 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CGAA 
 151 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 11292 
 1129202 
 30325 
 DEN 
 Denver, CO 
 CO 
 8 
 Colorado 
 82 
 1555 
 1600.0 
 5.0 
 5.0 
 0.0 
 0.0 
 1500-1559 
 10.0 
 1610.0 
 1732.0 
 12.0 
 ... 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 1 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3ELAA 
 1221 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 13204 
 1320402 
 31454 
 MCO 
 Orlando, FL 
 FL 
 12 
 Florida 
 33 
 1505 
 1605.0 
 60.0 
 60.0 
 1.0 
 4.0 
 1500-1559 
 12.0 
 1617.0 
 1927.0 
 5.0 
 ... 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 2 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3JSAA 
 985 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 13303 
 1330303 
 32467 
 MIA 
 Miami, FL 
 FL 
 12 
 Florida 
 33 
 1455 
 1525.0 
 30.0 
 30.0 
 1.0 
 2.0 
 1400-1459 
 10.0 
 1535.0 
 1908.0 
 18.0 
 ... 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 3 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3JXAA 
 41 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14747 
 1474703 
 30559 
 SEA 
 Seattle, WA 
 WA 
 53 
 Washington 
 93 
 2055 
 2330.0 
 155.0 
 155.0 
 1.0 
 10.0 
 2000-2059 
 28.0 
 2358.0 
 213.0 
 6.0 
 ... 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 4 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3KSAA 
 380 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 12953 
 1295302 
 31703 
 LGA 
 New York, NY 
 NY 
 36 
 New York 
 22 
 1040 
 1131.0 
 51.0 
 51.0 
 1.0 
 3.0 
 1000-1059 
 13.0 
 1144.0 
 1417.0 
 3.0 
 ... 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 N

In [20]:
airlines_eda.describe()

YEAR 
 QUARTER 
 MONTH 
 DAY_OF_MONTH 
 DAY_OF_WEEK 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN_STATE_FIPS 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST_STATE_FIPS 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ARR_DELAY_GROUP 
 CANCELLED 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 DISTANCE_GROUP 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 
 
 
 
 count 
 8125.0 
 8125.0 
 8125.000000 
 8125.000000 
 8125.000000 
 8125.000000 
 8125.000000 
 8125.000000 
 8.125000e+03 
 8125.000000 
 8125.000000 
 8125.000000 
 8125.000000 
 8.125000e+03 
 8125.000000 
 8125.000000 
 8125.000000 
 8125.000000 
 7916.000000 
 7916.000000 
 7916.000000 
 7916.000000 
 7916.000000 
 7910.000000 
 7910.000000 
 7899.000000 
 7899.000000 
 8125.000000 
 7899.000000 
 7882.000000 
 7882.000000 
 7882.000000 
 7882.000000 
 8125.000000 
 8125.000000 
 8125.000000 
 7882.00000 
 7882.000000 
 8125.0 
 8125.000000 
 8125.000000 
 1777.000000 
 1777.000000 
 1777.000000 
 1777.000000 
 1777.000000 
 66.000000 
 66.000000 
 66.000000 
 8125.000000 
 25.000000 
 17.000000 
 17.000000 
 25.000000 
 28.000000 
 2.800000e+01 
 28.000000 
 28.000000 
 28.000000 
 17.000000 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 0.0 
 
 
 mean 
 2015.0 
 1.0 
 2.024862 
 15.692554 
 3.911015 
 20009.208738 
 2510.420308 
 11948.041354 
 1.194808e+06 
 30651.628923 
 14.756062 
 37.073108 
 12673.507200 
 1.267353e+06 
 32010.278154 
 28.587077 
 48.710892 
 1428.507200 
 1445.036761 
 12.254548 
 14.058994 
 0.237999 
 0.247852 
 17.985588 
 1473.737168 
 1574.004178 
 6.151412 
 1591.112615 
 1577.000760 
 7.063563 
 13.766557 
 0.225450 
 -0.006724 
 0.026831 
 0.003077 
 123.752492 
 119.21492 
 95.091982 
 1.0 
 657.063385 
 3.105231 
 19.012943 
 6.443444 
 13.168824 
 0.042769 
 17.223410 
 1471.757576 
 34.212121 
 33.772727 
 0.006400 
 0.680000 
 357.529412 
 260.470588 
 28.880000 
 13113.750000 
 1.311378e+06 
 1529.214286 
 18.750000 
 13.214286 
 1567.470588 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 std 
 0.0 
 0.0 
 0.840050 
 8.742374 
 1.997680 
 316.410508 
 1741.531315 
 1753.419008 
 1.753409e+05 
 287.852540 
 1.985190 
 3.474082 
 1525.970901 
 1.525970e+05 
 1384.204975 
 16.104555 
 21.416809 
 450.306928 
 459.279060 
 34.131221 
 33.286439 
 0.425885 
 2.103984 
 8.933072 
 460.595725 
 481.702223 
 4.754134 
 463.145766 
 487.582458 
 37.264343 
 33.721962 
 0.417905 
 2.258698 
 0.161599 
 0.055388 
 63.344763 
 61.91198 
 60.444490 
 0.0 
 464.068155 
 1.832976 
 33.993696 
 28.149002 
 25.886766 
 1.192778 
 32.914203 
 441.802847 
 17.802779 
 17.125595 
 0.181515 
 0.476095 
 277.309421 
 271.148971 
 50.775913 
 1588.011303 
 1.588006e+05 
 487.867116 
 11.227034 
 7.375420 
 453.452467 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 min 
 2015.0 
 1.0 
 1.000000 
 1.000000 
 1.000000 
 19393.000000 
 6.000000 
 10397.000000 
 1.039705e+06 
 30397.000000 
 13.000000 
 34.000000 
 10135.000000 
 1.013503e+06 
 30135.000000 
 1.000000 
 1.000000 
 500.000000 
 15.000000 
 -18.000000 
 0.000000 
 0.000000 
 -2.000000 
 3.000000 
 4.000000 
 1.000000 

In [21]:
# Quick look at each of the columns data
airlines_eda.hist(figsize=(25,25), bins=110)

In [22]:
##Correlation Matrix
air_corr = airlines_eda.corr()
fig, ax = plt.subplots(figsize=(12, 12))
mask = np.zeros_like(air_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sn.diverging_palette(240, 10, as_cmap=True)
sn.heatmap(air_corr, mask=mask, cmap=cmap, center=0, linewidths=.5)
plt.title("Correlations between features.")
display(plt.show())

In [23]:
# Unique value counts of each column
for col in airlines_eda.columns:
  print(col, airlines_eda[col].nunique())

YEAR 1
QUARTER 1
MONTH 3
DAY_OF_MONTH 31
DAY_OF_WEEK 7
FL_DATE 90
OP_UNIQUE_CARRIER 13
OP_CARRIER_AIRLINE_ID 13
OP_CARRIER 13
TAIL_NUM 2513
OP_CARRIER_FL_NUM 2499
ORIGIN_AIRPORT_ID 2
ORIGIN_AIRPORT_SEQ_ID 2
ORIGIN_CITY_MARKET_ID 2
ORIGIN 2
ORIGIN_CITY_NAME 2
ORIGIN_STATE_ABR 2
ORIGIN_STATE_FIPS 2
ORIGIN_STATE_NM 2
ORIGIN_WAC 2
DEST_AIRPORT_ID 187
DEST_AIRPORT_SEQ_ID 187
DEST_CITY_MARKET_ID 171
DEST 187
DEST_CITY_NAME 183
DEST_STATE_ABR 51
DEST_STATE_FIPS 51
DEST_STATE_NM 51
DEST_WAC 51
CRS_DEP_TIME 795
DEP_TIME 1066
DEP_DELAY 228
DEP_DELAY_NEW 211
DEP_DEL15 2
DEP_DELAY_GROUP 15
DEP_TIME_BLK 19
TAXI_OUT 89
WHEELS_OFF 1075
WHEELS_ON 1071
TAXI_IN 54
CRS_ARR_TIME 986
ARR_TIME 1069
ARR_DELAY 264
ARR_DELAY_NEW 217
ARR_DEL15 2
ARR_DELAY_GROUP 15
ARR_TIME_BLK 19
CANCELLED 2
CANCELLATION_CODE 3
DIVERTED 2
CRS_ELAPSED_TIME 289
ACTUAL_ELAPSED_TIME 317
AIR_TIME 302
FLIGHTS 1
DISTANCE 279
DISTANCE_GROUP 10
CARRIER_DELAY 146
WEATHER_DELAY 99
NAS_DELAY 118
SECURITY_DELAY 4
LATE_AIRCRAFT_DELAY 138
FIRST_DEP_TIME 63
TOTAL_ADD_GTIME 43
LONGEST_ADD_GTIME 43
DIV_AIRPORT_LANDINGS 3
DIV_REACHED_DEST 2
DIV_ACTUAL_ELAPSED_TIME 17
DIV_ARR_DELAY 17
DIV_DISTANCE 8
DIV1_AIRPORT 16
DIV1_AIRPORT_ID 16
DIV1_AIRPORT_SEQ_ID 16
DIV1_WHEELS_ON 28
DIV1_TOTAL_GTIME 19
DIV1_LONGEST_GTIME 18
DIV1_WHEELS_OFF 17
DIV1_TAIL_NUM 17
DIV2_AIRPORT 0
DIV2_AIRPORT_ID 0
DIV2_AIRPORT_SEQ_ID 0
DIV2_WHEELS_ON 0
DIV2_TOTAL_GTIME 0
DIV2_LONGEST_GTIME 0
DIV2_WHEELS_OFF 0
DIV2_TAIL_NUM 0
DIV3_AIRPORT 0
DIV3_AIRPORT_ID 0
DIV3_AIRPORT_SEQ_ID 0
DIV3_WHEELS_ON 0
DIV3_TOTAL_GTIME 0
DIV3_LONGEST_GTIME 0
DIV3_WHEELS_OFF 0
DIV3_TAIL_NUM 0
DIV4_AIRPORT 0
DIV4_AIRPORT_ID 0
DIV4_AIRPORT_SEQ_ID 0
DIV4_WHEELS_ON 0
DIV4_TOTAL_GTIME 0
DIV4_LONGEST_GTIME 0
DIV4_WHEELS_OFF 0
DIV4_TAIL_NUM 0
DIV5_AIRPORT 0
DIV5_AIRPORT_ID 0
DIV5_AIRPORT_SEQ_ID 0
DIV5_WHEELS_ON 0
DIV5_TOTAL_GTIME 0
DIV5_LONGEST_GTIME 0
DIV5_WHEELS_OFF 0
DIV5_TAIL_NUM 0

In [24]:
# Check the depature airports
airlines_eda.ORIGIN.value_counts()

Out[17]: ATL 4558
ORD 3567
Name: ORIGIN, dtype: int64

In [25]:
# To check all the columns
airlines_eda.iloc[:, 0:40].head()

YEAR 
 QUARTER 
 MONTH 
 DAY_OF_MONTH 
 DAY_OF_WEEK 
 FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN 
 ORIGIN_CITY_NAME 
 ORIGIN_STATE_ABR 
 ORIGIN_STATE_FIPS 
 ORIGIN_STATE_NM 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST 
 DEST_CITY_NAME 
 DEST_STATE_ABR 
 DEST_STATE_FIPS 
 DEST_STATE_NM 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 
 
 
 
 0 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CGAA 
 151 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 11292 
 1129202 
 30325 
 DEN 
 Denver, CO 
 CO 
 8 
 Colorado 
 82 
 1555 
 1600.0 
 5.0 
 5.0 
 0.0 
 0.0 
 1500-1559 
 10.0 
 1610.0 
 1732.0 
 12.0 
 
 
 1 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3ELAA 
 1221 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 13204 
 1320402 
 31454 
 MCO 
 Orlando, FL 
 FL 
 12 
 Florida 
 33 
 1505 
 1605.0 
 60.0 
 60.0 
 1.0 
 4.0 
 1500-1559 
 12.0 
 1617.0 
 1927.0 
 5.0 
 
 
 2 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3JSAA 
 985 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 13303 
 1330303 
 32467 
 MIA 
 Miami, FL 
 FL 
 12 
 Florida 
 33 
 1455 
 1525.0 
 30.0 
 30.0 
 1.0 
 2.0 
 1400-1459 
 10.0 
 1535.0 
 1908.0 
 18.0 
 
 
 3 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3JXAA 
 41 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14747 
 1474703 
 30559 
 SEA 
 Seattle, WA 
 WA 
 53 
 Washington 
 93 
 2055 
 2330.0 
 155.0 
 155.0 
 1.0 
 10.0 
 2000-2059 
 28.0 
 2358.0 
 213.0 
 6.0 
 
 
 4 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3KSAA 
 380 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 12953 
 1295302 
 31703 
 LGA 
 New York, NY 
 NY 
 36 
 New York 
 22 
 1040 
 1131.0 
 51.0 
 51.0 
 1.0 
 3.0 
 1000-1059 
 13.0 
 1144.0 
 1417.0 
 3.0

In [26]:
airlines_eda.iloc[:, 40:80].head()

CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ARR_DELAY_GROUP 
 ARR_TIME_BLK 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 DISTANCE_GROUP 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 
 
 
 
 0 
 1730 
 1744.0 
 14.0 
 14.0 
 0.0 
 0.0 
 1700-1759 
 0.0 
 None 
 0.0 
 155.0 
 164.0 
 142.0 
 1.0 
 888.0 
 4 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 
 
 1 
 1835 
 1932.0 
 57.0 
 57.0 
 1.0 
 3.0 
 1800-1859 
 0.0 
 None 
 0.0 
 150.0 
 147.0 
 130.0 
 1.0 
 1005.0 
 5 
 6.0 
 0.0 
 0.0 
 0.0 
 51.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 
 
 2 
 1900 
 1926.0 
 26.0 
 26.0 
 1.0 
 1.0 
 1900-1959 
 0.0 
 None 
 0.0 
 185.0 
 181.0 
 153.0 
 1.0 
 1197.0 
 5 
 26.0 
 0.0 
 0.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 
 
 3 
 2325 
 219.0 
 174.0 
 174.0 
 1.0 
 11.0 
 2300-2359 
 0.0 
 None 
 0.0 
 270.0 
 289.0 
 255.0 
 1.0 
 1721.0 
 7 
 30.0 
 125.0 
 19.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 
 
 4 
 1340 
 1420.0 
 40.0 
 40.0 
 1.0 
 2.0 
 1300-1359 
 0.0 
 None 
 0.0 
 120.0 
 109.0 
 93.0 
 1.0 
 733.0 
 3 
 40.0 
 0.0 
 0.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN

In [27]:
airlines_eda.iloc[:, 80:].head()

DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 
 
 
 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 1 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 2 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 3 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 
 
 4 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None

#### Discussion of flight data variables

* Exclusion
  * The variables beginning with DIV2, DIV3, DIV4, and DIV5 are completely empty
  * All variables with SEQ_ID contain unique identifiers at a given point in time. For our purposes, this is less useful and redundant to the more generalizable fields with the same information applicable across time
  * ORIGIN, ORIGIN_CITY_NAME, ORIGIN_STATE_FIPS, ORIGIN_STATE_NM and the corresponding fields for DEST are all redundant with other fields that we will retain that identify airport ID, city, market and state

* Future Notes
  * There is a good amount of redundant data representation in Departure time and arrival time fields, though not sure which represention we will need yet, so leaving them in. Can revisit their exclusion later.

## Explore Weather Datasets

In [30]:
weather = spark.read.option("header", "true").parquet(f"dbfs:/mnt/mids-w261/data/datasets_final_project/weather_data/weather2015a.parquet/*.parquet")
#only Q1 2015 for EDA
weather = weather.filter(weather["DATE"]<(lit("2015-04-01"))) 

In [31]:
weather.count()

Out[22]: 29643209

In [32]:
weather.printSchema()

root
-- STATION: string (nullable = true)
-- DATE: timestamp (nullable = true)
-- SOURCE: short (nullable = true)
-- LATITUDE: double (nullable = true)
-- LONGITUDE: double (nullable = true)
-- ELEVATION: double (nullable = true)
-- NAME: string (nullable = true)
-- REPORT_TYPE: string (nullable = true)
-- CALL_SIGN: string (nullable = true)
-- QUALITY_CONTROL: string (nullable = true)
-- WND: string (nullable = true)
-- CIG: string (nullable = true)
-- VIS: string (nullable = true)
-- TMP: string (nullable = true)
-- DEW: string (nullable = true)
-- SLP: string (nullable = true)
-- AW1: string (nullable = true)
-- GA1: string (nullable = true)
-- GA2: string (nullable = true)
-- GA3: string (nullable = true)
-- GA4: string (nullable = true)
-- GE1: string (nullable = true)
-- GF1: string (nullable = true)
-- KA1: string (nullable = true)
-- KA2: string (nullable = true)
-- MA1: string (nullable = true)
-- MD1: string (nullable = true)
-- MW1: string (nullable = true)
-- MW2: string (nullable = true)
-- OC1: string (nullable = true)
-- OD1: string (nullable = true)
-- OD2: string (nullable = true)
-- REM: string (nullable = true)
-- EQD: string (nullable = true)
-- AW2: string (nullable = true)
-- AX4: string (nullable = true)
-- GD1: string (nullable = true)
-- AW5: string (nullable = true)
-- GN1: string (nullable = true)
-- AJ1: string (nullable = true)
-- AW3: string (nullable = true)
-- MK1: string (nullable = true)
-- KA4: string (nullable = true)
-- GG3: string (nullable = true)
-- AN1: string (nullable = true)
-- RH1: string (nullable = true)
-- AU5: string (nullable = true)
-- HL1: string (nullable = true)
-- OB1: string (nullable = true)
-- AT8: string (nullable = true)
-- AW7: string (nullable = true)
-- AZ1: string (nullable = true)
-- CH1: string (nullable = true)
-- RH3: string (nullable = true)
-- GK1: string (nullable = true)
-- IB1: string (nullable = true)
-- AX1: string (nullable = true)
-- CT1: string (nullable = true)
-- AK1: string (nullable = true)
-- CN2: string (nullable = true)
-- OE1: string (nullable = true)
-- MW5: string (nullable = true)
-- AO1: string (nullable = true)
-- KA3: string (nullable = true)
-- AA3: string (nullable = true)
-- CR1: string (nullable = true)
-- CF2: string (nullable = true)
-- KB2: string (nullable = true)
-- GM1: string (nullable = true)
-- AT5: string (nullable = true)
-- AY2: string (nullable = true)
-- MW6: string (nullable = true)
-- MG1: string (nullable = true)
-- AH6: string (nullable = true)
-- AU2: string (nullable = true)
-- GD2: string (nullable = true)
-- AW4: string (nullable = true)
-- MF1: string (nullable = true)
-- AA1: string (nullable = true)
-- AH2: string (nullable = true)
-- AH3: string (nullable = true)
-- OE3: string (nullable = true)
-- AT6: string (nullable = true)
-- AL2: string (nullable = true)
-- AL3: string (nullable = true)
-- AX5: string (nullable = true)
-- IB2: string (nullable = true)
-- AI3: string (nullable = true)
-- CV3: string (nullable = true)
-- WA1: string (nullable = true)
-- GH1: string (nullable = true)
-- KF1: string (nullable = true)
-- CU2: string (nullable = true)
-- CT3: string (nullable = true)
-- SA1: string (nullable = true)
-- AU1: string (nullable = true)
-- KD2: string (nullable = true)
-- AI5: string (nullable = true)
-- GO1: string (nullable = true)
-- GD3: string (nullable = true)
-- CG3: string (nullable = true)
-- AI1: string (nullable = true)
-- AL1: string (nullable = true)
-- AW6: string (nullable = true)
-- MW4: string (nullable = true)
-- AX6: string (nullable = true)
-- CV1: string (nullable = true)
-- ME1: string (nullable = true)
-- KC2: string (nullable = true)
-- CN1: string (nullable = true)
-- UA1: string (nullable = true)
-- GD5: string (nullable = true)
-- UG2: string (nullable = true)
-- AT3: string (nullable = true)
-- AT4: string (nullable = true)
-- GJ1: string (nullable = true)
-- MV1: string (nullable = true)
-- GA5: string (nullable = true)
-- CT2: string (nullable = true)
-- CG2: string

## Split the Weather dataset
- weather_eda (5%)
- weather_other (95%)

In [34]:
weather_eda, weather_other = weather.repartition(20,'DATE').randomSplit([0.05,0.95], seed=4)
weather_eda = weather_eda.toPandas()

In [35]:
weather_eda.shape

Out[25]: (1481220, 177)

In [36]:
weather_eda.head()

STATION 
 DATE 
 SOURCE 
 LATITUDE 
 LONGITUDE 
 ELEVATION 
 NAME 
 REPORT_TYPE 
 CALL_SIGN 
 QUALITY_CONTROL 
 WND 
 CIG 
 VIS 
 TMP 
 DEW 
 SLP 
 AW1 
 GA1 
 GA2 
 GA3 
 GA4 
 GE1 
 GF1 
 KA1 
 KA2 
 MA1 
 MD1 
 MW1 
 MW2 
 OC1 
 OD1 
 OD2 
 REM 
 EQD 
 AW2 
 AX4 
 GD1 
 AW5 
 GN1 
 AJ1 
 ... 
 GA6 
 KG1 
 AU3 
 AT7 
 KD1 
 GL1 
 IA1 
 GG2 
 OD3 
 UG1 
 CB1 
 AI6 
 CI1 
 CV2 
 AZ2 
 AD1 
 AH1 
 WD1 
 AA4 
 KC1 
 IA2 
 CF3 
 AI2 
 AT1 
 GD4 
 AX3 
 AH4 
 KB3 
 CU1 
 CN4 
 AT2 
 CG1 
 CF1 
 GG1 
 MV2 
 CW1 
 GG4 
 AB1 
 AH5 
 CN3 
 
 
 
 
 0 
 None 
 2015-01-01 11:36:00 
 6.0 
 52.22028 
 -174.20611 
 17.1 
 ATKA AIRPORT, AK US 
 FM-15 
 PAAK 
 V020 
 999,9,C,0000,5 
 00914,5,M,N 
 016093,5,N,5 
 +0030,5 
 -0010,5 
 99999,9 
 
 04,5,+00640,5,99,9 
 07,5,+00914,5,99,9 
 08,5,+01433,5,99,9 
 
 
 
 
 
 10125,5,10105,5 
 
 
 
 
 
 
 MET09101/01/15 01:36:02 METAR PAAK 011136Z 000... 
 
 
 
 2,99,1,+00640,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 None 
 2015-01-01 12:33:00 
 6.0 
 42.93800 
 -85.06100 
 249.0 
 IONIA CO AIRPORT, MI US 
 FM-15 
 KY70 
 V020 
 260,5,N,0098,5 
 00549,5,M,N 
 016093,5,N,5 
 -0045,5 
 -0080,5 
 99999,9 
 
 08,5,+00549,5,99,9 
 
 
 
 
 
 
 
 10122,5,09826,5 
 
 
 
 0129,5 
 
 
 MET09101/01/15 07:33:02 METAR KY70 011233Z 260... 
 
 
 
 4,99,1,+00549,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 None 
 2015-01-01 18:50:00 
 6.0 
 40.98300 
 -122.69400 
 728.2 
 TRINITY CENTER AIRPORT, CA US 
 FM-15 
 KO86 
 V020 
 999,9,C,0000,5 
 22000,5,9,N 
 016093,5,N,5 
 +0000,C 
 -0030,C 
 99999,9 
 
 00,5,+99999,9,99,9 
 
 
 
 
 
 
 
 10224,5,09372,5 
 
 
 
 
 
 
 MET07401/01/15 10:50:02 METAR KO86 011850Z 000... 
 R01 007TMP046R02 M037DPT046 
 
 
 0,99,1,+99999,9,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 None 
 2015-01-01 19:33:00 
 6.0 
 41.51627 
 -82.86948 
 179.8 
 CARL R KELLER FIELD AIRPORT, OH US 
 FM-15 
 KPCW 
 V020 
 200,5,N,0118,5 
 22000,5,9,N 
 016093,5,N,5 
 +0010,5 
 -0110,5 
 99999,9 
 
 00,5,+99999,9,99,9 
 
 
 
 
 
 
 
 10135,5,09921,5 
 
 
 
 
 
 
 MET09201/01/15 14:33:02 METAR KPCW 011933Z 200... 
 
 
 
 0,99,1,+99999,9,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4 
 None 
 2015-01-01 20:35:00 
 6.0 
 34.14694 
 -97.12250 
 257.3 
 ARDMORE DOWNTOWN AIRPORT, OK US 
 FM-15 
 K1F0 
 V020 
 999,9,C,0000,5 
 00122,5,M,N 
 006437,5,N,5 
 +0000,5 
 +0000,5 
 99999,9 
 10,5 
 08,5,+00122,5,99,9 
 
 
 
 
 
 
 
 10237,5,09929,5 
 
 
 
 
 
 
 MET07801/01/15 14:35:02 METAR K1F0 012035Z 000... 
 
 
 
 4,99,1,+00122,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 rows × 177 columns

## Weather columns
- AA - liquid precipitation - depth (milimeters - index 1) - USE THIS ONE
- AB - liquid precipitation monthly total - depth (milimeters - index 0)
- AD - liquid precipitation greatest amount in 24 hours for the month (milimeters - index 0)
- AE - liquid precipitation number of days with specific amount 
- AH - liquid precipitation max short duration for the month
- AI - identical to above
- AJ - snow depth - centimeters - index 0 - USE THIS ONE
- AK - snow depth, greatest depth on the ground for the month - depth (centimeters - index 0)
- AM - snow depth, greatest amount in 24 hour period - depth (centimeters - index 0)
- AT - Present weather observation - categorical - index 1 - USE THIS ONE
- AW - Present weather observation - categorical - index 0
- AX - Past weather observation - categorical - index 0 - USE THIS ONE
- ED - Runway visual range - meters - index 2 - USE THIS ONE
- GA - Sky cover layer - Height of lowest surface of a cloud (meters - index2)
- GD - Sky cover summation - categorical - index 0 - USE THIS ONE
- GF - Sky condition observation - total coverage code - categorical
- KA - Extreme air temperature - celsius - index 2 - USE THIS ONE
- KB - Average air temperature - celsius - index 2 - USE THIS ONE
- KC - Extreme air temperature for the month - celsius - index 2
- KD - Heating cooling degree days
- KG - Average Dew Point - celsius - index 2
- MA - Atmospheric pressure - hectopascals - index 0 - USE THIS ONE
- MD - Atmospheric pressure Change three hour quantity - hectopascals - index 2 - USE THIS ONE
- MF - Atmospheric pressure average station pressure for the day - hectopascals - index 0 - USE THIS ONE
- MG - Atmospheric pressure average station pressure for the day
- MH - Atmospheric pressure average station pressure for the month
- MK - Atmospheric pressure for the month
- MW - present weather observation 
- OC - Wind Gust - meters per second - index 0 - USE THIS ONE
- OD - Supplementary wind observatoin
- OE - Summary of the day Wind - meters per second - index 0 - USE THIS ONE
- RH - Relative Humidity - percent - index 2 - USE THIS ONE
- WA - Ice Accretion - centimeters - index 1 - USE THIS ONE

## Other columns to use :
- AA - Liquid Precipitation - The identifier that represents an episode of LIQUID-PRECIPITATION - depth in millimeters
- AJ - Snow Depth - The identifier that denotes the start of a SNOW-DEPTH data section - depth in centimeters
- AT - Present Weather Observation - The identifier that signifies the reporting of present weather - Categorical
> Categories
>> 01 = Fog, ice fog or freezing fog (may include heavy fog) 
>> 02 = Heavy fog or heavy freezing fog (not always distinguished from fog)
>> 03 = Thunder
>> 04 = Ice pellets, sleet, snow pellets or small hail
>> 05 = Hail (may include small hail)
>> 06 = Glaze or rime
>> 07 = Dust, volcanic ash, blowing dust, blowing sand or blowing obstruction
>> 08 = Smoke or haze
>> 09 = Blowing or drifting snow
>> 10 = Tornado, water spout or funnel cloud
>> 11 = High or damaging winds
>> 12 = Blowing spray
>> 13 = Mist
>> 14 = Drizzle
>> 15 = Freezing drizzle
>> 16 = Rain
>> 17 = Freezing rain
>> 18 = Snow, snow pellets, snow grains or ice crystals
>> 19 = Unknown precipitation
>> 21 = Ground fog
>> 22 = Ice fog or freezing fog
- AX - Past Weather Observation - The identifier that signifies the reporting of past weather as summarized for the calendar day - Categorical
> Categories
>> 00 = none to report 
>> 01 = fog
>> 02 = fog reducing visibility to ¼ mile or less
>> 03 = thunder
>> 04 = ice pellets
>> 05 = hail
>> 06 = glaze or rime
>> 07 = blowing dust or sand, visibility ½ mile or less
>> 08 = smoke or haze
>> 09 = blowing snow
>> 10 = tornado
>> 11 = high or damaging winds
>> 99 = missing
- ED - Runway Visual Range - The dimension of the horizontal distance that can be seen along the runway - meters
- GD - Sky Cover Summation - The code that denotes the portion of the total celestial dome covered by all layers of clouds and other - Categorical
> Categories
>> 0 = Clear - No coverage
 1 = FEW - 2/8 or less coverage (not including zero)
 2 = SCATTERED - 3/8-4/8 coverage
 3 = BROKEN - 5/8-7/8 coverage
 4 = OVERCAST - 8/8 coverage
 5 = OBSCURED
 6 = PARTIALLY OBSCURED
 9 = MISSING
- KA - Extreme Air Temperature - The temperature of the high or low air temperature for a given period - degrees celsius
- KB - Average Air Temperature - The mean air temperature for a given period, typically for the day or month, as reported by the station - degrees celsius
- MA - Atmospheric Pressure Observation - The atmospheric pressure at the observation point - hectopascals
- MD - Atmospheric Pressure Change - The absolute value of the quantity of change in atmospheric pressure measured at the beginning and end of a three hour period - hectopascals
- MF - Atmospheric Pressure Observation - The average pressure at the observed point for the day derived computationally from other QC’ed elements - hectopascals
- OC - Wind Gust - The rate of speed of a wind gust - meters per second
- OE - Summary of Day Wind Observation - The rate of horizontal wind speed of air reported in the SUMMARY-OF-DAY-WIND-OBSERVATION - meters per second
- RH - Relative Humidity - The average maximum or minimum relative humidity for a given period, typically for the day or month, derived from other data fields - percent
- WA - Ice Accretion - The thickness of the ice that has accumulated on a marine platform - centimeters

In [39]:
weather_eda.hist(figsize=(25,25), bins=110)

## Explore Station Datasets

In [41]:
stations = spark.read.option("header", "true").csv("dbfs:/mnt/mids-w261/data/DEMO8/gsod/stations.csv.gz")

In [42]:
stations = stations.toPandas()

In [43]:
stations.head()

usaf 
 wban 
 name 
 country 
 state 
 call 
 lat 
 lon 
 elev 
 begin 
 end 
 
 
 
 
 0 
 007018 
 99999 
 WXPOD 7018 
 None 
 None 
 None 
 0 
 0 
 +7018.0 
 20110309 
 20130730 
 
 
 1 
 007026 
 99999 
 WXPOD 7026 
 AF 
 None 
 None 
 0 
 0 
 +7026.0 
 20120713 
 20170822 
 
 
 2 
 007070 
 99999 
 WXPOD 7070 
 AF 
 None 
 None 
 0 
 0 
 +7070.0 
 20140923 
 20150926 
 
 
 3 
 008268 
 99999 
 WXPOD8278 
 AF 
 None 
 None 
 32.95 
 65.567 
 +1156.7 
 20100519 
 20120323 
 
 
 4 
 008307 
 99999 
 WXPOD 8318 
 AF 
 None 
 None 
 0 
 0 
 +8318.0 
 20100421 
 20100421

In [44]:
stations[stations['state']=='IL']

usaf 
 wban 
 name 
 country 
 state 
 call 
 lat 
 lon 
 elev 
 begin 
 end 
 
 
 
 
 6364 
 720103 
 99999 
 WILMETTE MARINES 
 US 
 IL 
 None 
 42.07 
 -87.68 
 +0183.0 
 19960101 
 19960214 
 
 
 6367 
 720141 
 04868 
 MARSHALL CO 
 US 
 IL 
 KC75 
 41.019 
 -89.386 
 +0173.1 
 20060101 
 20130430 
 
 
 6770 
 722417 
 99999 
 WILMETTE (CGS) 
 US 
 IL 
 None 
 42.067 
 -87.683 
 +0183.0 
 19870501 
 19910207 
 
 
 6771 
 722418 
 99999 
 CALUMET (CGS) 
 US 
 IL 
 63G 
 41.717 
 -87.533 
 +0180.0 
 19870501 
 19900810 
 
 
 7000 
 724337 
 03838 
 VANDALIA MUNICIPAL 
 US 
 IL 
 KVLA 
 39 
 -89.167 
 +0164.0 
 19730101 
 19730515 
 
 
 7009 
 724394 
 99999 
 ALTON 
 US 
 IL 
 None 
 38.883 
 -90.05 
 +0166.0 
 19840225 
 19840229 
 
 
 7010 
 724396 
 93989 
 QUINCY RGNL BALDWIN 
 US 
 IL 
 KUIN 
 39.937 
 -91.192 
 +0233.5 
 19730101 
 20100801 
 
 
 7011 
 724398 
 99999 
 WEST PLAINS 
 US 
 IL 
 None 
 36.883 
 -91.9 
 +0374.0 
 19760402 
 20010516 
 
 
 7131 
 725306 
 14855 
 GLENVIEW NAS 
 US 
 IL 
 KNBU 
 42.083 
 -87.833 
 +0199.0 
 19730101 
 19950301 
 
 
 7132 
 725306 
 99999 
 GLENVIEW NAS 
 US 
 IL 
 KNBU 
 42.083 
 -87.817 
 +0199.0 
 20050905 
 20050905 
 
 
 7133 
 725307 
 99999 
 WILMETTE (MARINES)& 
 US 
 IL 
 K62G 
 42.067 
 -87.683 
 +0223.0 
 19760212 
 19960214 
 
 
 7134 
 725310 
 14806 
 RANTOUL/CHANUTE AFB 
 US 
 IL 
 None 
 40.3 
 -88.15 
 +0228.0 
 19360701 
 19710528 
 
 
 7135 
 725310 
 99999 
 RANTOUL 
 US 
 IL 
 None 
 40.3 
 -88.15 
 +0228.0 
 20020228 
 20041006 
 
 
 7136 
 725323 
 99999 
 GALESBURG MUNICIPAL 
 US 
 IL 
 GBG 
 40.933 
 -90.433 
 +0245.0 
 19810904 
 19950814 
 
 
 7137 
 725325 
 14816 
 BRADFORD/RINKENBERG 
 US 
 IL 
 KBDF 
 41.233 
 -89.617 
 +0247.0 
 19730101 
 19751214 
 
 
 7138 
 725325 
 99999 
 BRADFORD/RINKENBERG 
 US 
 IL 
 KBDF 
 41.233 
 -89.617 
 +0247.0 
 20011226 
 20011226 
 
 
 7140 
 725339 
 99999 
 CHIGAGO-WAUKEE ARPT 
 US 
 IL 
 KPWK 
 42.117 
 -87.9 
 +0197.0 
 20010811 
 20020621 
 
 
 7141 
 725342 
 99999 
 LAWRENCEVILLE VINCEN 
 US 
 IL 
 KLWV 
 38.764 
 -87.605 
 +0131.0 
 19970123 
 20051230 
 
 
 7143 
 725346 
 94866 
 CHICAGO/MEIGS 
 US 
 IL 
 KCGX 
 41.867 
 -87.6 
 +0181.0 
 19730101 
 20080618 
 
 
 7144 
 725346 
 99999 
 CHICAGO/MEIGS 
 US 
 IL 
 KCGX 
 41.867 
 -87.6 
 +0181.0 
 20000101 
 20030409 
 
 
 7474 
 744661 
 99999 
 EFFINGHAM CO MEM 
 US 
 IL 
 None 
 39.07 
 -88.534 
 +0178.9 
 20020603 
 20130430 
 
 
 7593 
 749075 
 99999 
 LAWERENCEVILLE IL 
 US 
 IL 
 LMV 
 38.766 
 -87.6 
 +0131.0 
 19430111 
 19460201 
 
 
 9902 
 999999 
 03838 
 VANDALIA FAA AIRPORT 
 US 
 IL 
 VLA 
 38.983 
 -89.167 
 +0163.1 
 19510401 
 19550101 
 
 
 10142 
 999999 
 14810 
 PARK RIDGE AF 
 US 
 IL 
 None 
 41.983 
 -87.9 
 +0203.0 
 19490101 
 19581101 
 
 
 10147 
 999999 
 14816 
 BRADFORD 
 US 
 IL 
 BDF 
 41.233 
 -89.617 
 +0242.0 
 19481201 
 19550101 
 
 
 10156 
 999999 
 14834 
 JOLIET 
 US 
 IL 
 KJOT 
 41.5 
 -88.167 
 +0177.1 
 19460101 
 19530315 
 
 
 10161 
 999999 
 14844 
 ROCKFORD CAA AIRPORT 
 US 
 IL 
 None 
 42.35 
 -89.05 
 +0225.9 
 19481201 
 19510301 
 
 
 10586 
 999999 
 93989 
 QUINCY MUNI BALDWIN FLD 
 US 
 IL 
 KUIN 
 39.937 
 -91.192 
 +0233.5 
 19481201 
 19550101 
 
 
 10628 
 999999 
 94866 
 CHICAGO MEIGS FIELD 
 US 
 IL 
 CGX 
 41.867 
 -87.617 
 +0180.1 
 19660101 
 19700701 
 
 
 14768 
 999999 
 14819 
 CHICAGO MIDWAY AP 
 US 
 IL 
 KMDW 
 41.786 
 -87.752 
 +0188.4 
 19480101 
 19721231 
 
 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 ... 
 
 
 28688 
 724975 
 93809 
 CAIRO REGIONAL AIRPORT 
 US 
 IL 
 KCIR 
 37.064 
 -89.219 
 +0097.8 
 20060101 
 20190304 
 
 
 28703 
 725300 
 94846 
 CHICAGO O'HARE INTERNATIONAL 
 US 
 IL 
 KORD 
 41.995 
 -87.934 
 +0201.8 
 19461001 
 20190304 
 
 
 28704 
 725305 
 94892 
 DUPAGE AIRPORT 
 US 
 IL 
 KDPA 
 41.914 
 -88.246 
 +0229.8 
 20060101 
 20190304 
 
 
 28705 
 725314 
 03960 
 ST LOUIS DOWNTOWN AIRPORT 
 US 
 IL 
 KCPS 
 38.57

# Feature Engineering
Apply relevant feature transformations, dimensionality reduction if needed, interaction terms, treatment of categorical variables, etc.. Justify your choices.
- What are the proposed features, and how are they derived from underlying data? 
- Do you need any dimensionality reduction? (ie L1, forward/backward selection, PCA, etc..)
- Specify the feature transformations for the pipeline and justify these features given the target (ie, hashing trick, tf-idf, stopword removal, lemmatization, tokenization, etc..)
- Other feature engineering efforts, ie interaction terms, Brieman’s method, etc…)

### 1. Create a new out come variable "DELAY_15"
- 1: Departure delay over 15 min
- 0: Departure not delay or delay less than 15 min

#### 1-1. First set the DELAY_15 as DEP_DEL15, change NaN value to 0

In [48]:
# Total counts of airlines is 161057
df_airlines.count()[0]

Out[32]: 161057

In [49]:
# There are NaN in DEP_DEL15. 119546 + 36685 != 161057
df_airlines.DEP_DEL15.value_counts()

Out[33]: 0.0 119546
1.0 36685
Name: DEP_DEL15, dtype: int64

In [50]:
# DEP_DEL15: Fill in O in NaN value
df_airlines['DEP_DEL15'] = df_airlines['DEP_DEL15'].fillna(0)
df_airlines.DEP_DEL15.value_counts()

Out[34]: 0.0 124372
1.0 36685
Name: DEP_DEL15, dtype: int64

In [51]:
# Assign the DELAY_15 as DEP_DEL15
df_airlines['DELAY_15'] = df_airlines.DEP_DEL15
print(df_airlines.DELAY_15.value_counts())
df_airlines.head()

YEAR 
 QUARTER 
 MONTH 
 DAY_OF_MONTH 
 DAY_OF_WEEK 
 FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN 
 ORIGIN_CITY_NAME 
 ORIGIN_STATE_ABR 
 ORIGIN_STATE_FIPS 
 ORIGIN_STATE_NM 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST 
 DEST_CITY_NAME 
 DEST_STATE_ABR 
 DEST_STATE_FIPS 
 DEST_STATE_NM 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015 
 1 
 1 
 1 
 4 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1040.0 
 5.0 
 5.0 
 0.0 
 0.0 
 1000-1059 
 15.0 
 1055.0 
 1419.0 
 6.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015 
 1 
 1 
 2 
 5 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1051.0 
 16.0 
 16.0 
 1.0 
 1.0 
 1000-1059 
 9.0 
 1100.0 
 1432.0 
 5.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015 
 1 
 1 
 3 
 6 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1041.0 
 6.0 
 6.0 
 0.0 
 0.0 
 1000-1059 
 22.0 
 1103.0 
 1437.0 
 20.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015 
 1 
 1 
 4 
 7 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1107.0 
 32.0 
 32.0 
 1.0 
 2.0 
 1000-1059 
 23.0 
 1130.0 
 1519.0 
 4.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1058.0 
 23.0 
 23.0 
 1.0 
 1.0 
 1000-1059 
 21.0 
 1119.0 
 1444.0 
 5.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 

#### 1-2. Handle exception for DELAY_15 in Canceled flight

In [53]:
# Check the canceled flight count
df_airlines.CANCELLED.value_counts()
# 218 canceled flights

Out[36]: 0.0 156073
1.0 4984
Name: CANCELLED, dtype: int64

In [54]:
# Get the rows of canceled flights
cancel_index = df_airlines.loc[df_airlines['CANCELLED'] == 1].index

In [55]:
# Some canceled flights had been labeled as delay flights, which should be labeled as non-delay flights
df_airlines.iloc[cancel_index,-1].value_counts()

Out[38]: 0.0 4882
1.0 102
Name: DELAY_15, dtype: int64

In [56]:
# When the flight is canceled assign DELAY_15 as 0
df_airlines.iloc[cancel_index,-1] = 0
df_airlines.iloc[cancel_index,-1].value_counts()

Out[39]: 0.0 4984
Name: DELAY_15, dtype: int64

#### 1-3. Check if there is an exception in DELAY_15 at diverted flight
Divert flight affected the arrivial time not departure time

In [58]:
# Check the canceled flight count
df_airlines.DIVERTED.value_counts()
# 25 diverted flights

Out[40]: 0.0 160676
1.0 381
Name: DIVERTED, dtype: int64

In [59]:
# Diverted flights does not need to change the values of DELAY_15, the values is same as DEP_DEL15
df_airlines.loc[df_airlines['DIVERTED'] == 1]

YEAR 
 QUARTER 
 MONTH 
 DAY_OF_MONTH 
 DAY_OF_WEEK 
 FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN 
 ORIGIN_CITY_NAME 
 ORIGIN_STATE_ABR 
 ORIGIN_STATE_FIPS 
 ORIGIN_STATE_NM 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST 
 DEST_CITY_NAME 
 DEST_STATE_ABR 
 DEST_STATE_FIPS 
 DEST_STATE_NM 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 3523 
 2015 
 1 
 1 
 26 
 1 
 2015-01-26 
 DL 
 19790 
 DL 
 N826DN 
 1865 
 10397 
 1039705 
 30397 
 ATL 
 Atlanta, GA 
 GA 
 13 
 Georgia 
 34 
 13303 
 1330303 
 32467 
 MIA 
 Miami, FL 
 FL 
 12 
 Florida 
 33 
 955 
 958.0 
 3.0 
 3.0 
 0.0 
 0.0 
 0900-0959 
 20.0 
 1018.0 
 1342.0 
 5.0 
 ... 
 14027.0 
 1402702.0 
 1209.0 
 17.0 
 13.0 
 1317.0 
 N826DN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 4109 
 2015 
 1 
 1 
 31 
 6 
 2015-01-31 
 DL 
 19790 
 DL 
 N6713Y 
 1172 
 10397 
 1039705 
 30397 
 ATL 
 Atlanta, GA 
 GA 
 13 
 Georgia 
 34 
 11503 
 1150303 
 31503 
 EGE 
 Eagle, CO 
 CO 
 8 
 Colorado 
 82 
 1015 
 1013.0 
 -2.0 
 0.0 
 0.0 
 -1.0 
 1000-1059 
 15.0 
 1028.0 
 1418.0 
 4.0 
 ... 
 11292.0 
 1129202.0 
 1229.0 
 38.0 
 34.0 
 1346.0 
 N6713Y 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 4257 
 2015 
 1 
 1 
 31 
 6 
 2015-01-31 
 DL 
 19790 
 DL 
 N6704Z 
 1757 
 10397 
 1039705 
 30397 
 ATL 
 Atlanta, GA 
 GA 
 13 
 Georgia 
 34 
 14908 
 1490803 
 32575 
 SNA 
 Santa Ana, CA 
 CA 
 6 
 California 
 91 
 956 
 1001.0 
 5.0 
 5.0 
 0.0 
 0.0 
 0900-0959 
 14.0 
 1015.0 
 1457.0 
 4.0 
 ... 
 12892.0 
 1289203.0 
 1142.0 
 30.0 
 21.0 
 1434.0 
 N6704Z 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 4482 
 2015 
 1 
 1 
 3 
 6 
 2015-01-03 
 EV 
 20366 
 EV 
 N931EV 
 5143 
 10397 
 1039705 
 30397 
 ATL 
 Atlanta, GA 
 GA 
 13 
 Georgia 
 34 
 14574 
 1457402 
 34574 
 ROA 
 Roanoke, VA 
 VA 
 51 
 Virginia 
 38 
 905 
 900.0 
 -5.0 
 0.0 
 0.0 
 -1.0 
 0900-0959 
 27.0 
 927.0 
 NaN 
 NaN 
 ... 
 10397.0 
 1039705.0 
 1016.0 
 21.0 
 13.0 
 1205.0 
 N931EV 
 GSO 
 11995.0 
 1199502.0 
 1345.0 
 3.0 
 3.0 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 4761 
 2015 
 1 
 1 
 5 
 1 
 2015-01-05 
 EV 
 20366 
 EV 
 N686BR 
 5247 
 10397 
 1039705 
 30397 
 ATL 
 Atlanta, GA 
 GA 
 13 
 Georgia 
 34 
 10781 
 1078102 
 30781 
 BTR 
 Baton Rouge, LA 
 LA 
 22 
 Louisiana 
 72 
 2149 
 2146.0 
 -3.0 
 0.0 
 0.0 
 -1

The new outcame variables is "DELAY_15"
- 1: Departure delay over 15 min
- 0: Departure not delay or delay less than 15 min

### 2. Feature reduction: remove highly correlated variables and redundant variables
Flight variables document: https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236

#### 2-1. Remove redundant variables

##### Time period variables: 
- YEAR	
- QUARTER	
- MONTH	
- DAY_OF_MONTH	
- DAY_OF_WEEK	
- **FL_DATE**

These variables all represent the time period, so we can choose only FL_DATE variable to represent all these time period variables.

In [64]:
# delete time period redundant variables
del_list_time_period = ['YEAR', 'QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK']

for col in del_list_time_period:
  del df_airlines[col]

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN_AIRPORT_ID 
 ORIGIN_AIRPORT_SEQ_ID 
 ORIGIN_CITY_MARKET_ID 
 ORIGIN 
 ORIGIN_CITY_NAME 
 ORIGIN_STATE_ABR 
 ORIGIN_STATE_FIPS 
 ORIGIN_STATE_NM 
 ORIGIN_WAC 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST 
 DEST_CITY_NAME 
 DEST_STATE_ABR 
 DEST_STATE_FIPS 
 DEST_STATE_NM 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1040.0 
 5.0 
 5.0 
 0.0 
 0.0 
 1000-1059 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 10.0 
 0.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1051.0 
 16.0 
 16.0 
 1.0 
 1.0 
 1000-1059 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 22.0 
 1.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1041.0 
 6.0 
 6.0 
 0.0 
 0.0 
 1000-1059 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 42.0 
 1.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1107.0 
 32.0 
 32.0 
 1.0 
 2.0 
 1000-1059 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 68.0 
 1.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 13930 
 1393003 
 30977 
 ORD 
 Chicago, IL 
 IL 
 17 
 Illinois 
 41 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1058.0 
 23.0 
 23.0 
 1.0 
 1.0 
 1000-1059 
 21.0 
 1119.0 
 1444.0 
 5.0 


##### Origin related variables: 
- ORIGIN_AIRPORT_ID	
- ORIGIN_AIRPORT_SEQ_ID	
- ORIGIN_CITY_MARKET_ID	
- **ORIGIN**	
- ORIGIN_CITY_NAME	
- ORIGIN_STATE_ABR	
- ORIGIN_STATE_FIPS	
- ORIGIN_STATE_NM	
- ORIGIN_WAC

These variables all represent the origin airport, so we can choose only ORIGIN variable to represent all these origin related variables.

In [66]:
# delete origin related redundant variables
del_list_origin = ['ORIGIN_AIRPORT_ID', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN_CITY_MARKET_ID', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_ABR', 'ORIGIN_STATE_FIPS', 'ORIGIN_STATE_NM', 'ORIGIN_WAC']

for col in del_list_origin:
  del df_airlines[col]

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST_AIRPORT_ID 
 DEST_AIRPORT_SEQ_ID 
 DEST_CITY_MARKET_ID 
 DEST 
 DEST_CITY_NAME 
 DEST_STATE_ABR 
 DEST_STATE_FIPS 
 DEST_STATE_NM 
 DEST_WAC 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ARR_DELAY_GROUP 
 ARR_TIME_BLK 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1040.0 
 5.0 
 5.0 
 0.0 
 0.0 
 1000-1059 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 10.0 
 0.0 
 0.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1051.0 
 16.0 
 16.0 
 1.0 
 1.0 
 1000-1059 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 22.0 
 1.0 
 1.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1041.0 
 6.0 
 6.0 
 0.0 
 0.0 
 1000-1059 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 42.0 
 1.0 
 2.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1107.0 
 32.0 
 32.0 
 1.0 
 2.0 
 1000-1059 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 68.0 
 1.0 
 4.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 14635 
 1463502 
 31714 
 RSW 
 Fort Myers, FL 
 FL 
 12 
 Florida 
 33 
 1035 
 1058.0 
 23.0 
 23.0 
 1.0 
 1.0 
 1000-1059 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 34.0 
 34.0 
 1.0 
 2.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 ... 
 NaN 
 NaN 


##### Destination related variables: 
- DEST_AIRPORT_ID	
- DEST_AIRPORT_SEQ_ID	
- DEST_CITY_MARKET_ID	
- **DEST**	
- DEST_CITY_NAME	
- DEST_STATE_ABR	
- DEST_STATE_FIPS	
- DEST_STATE_NM	
- DEST_WAC	

These variables all represent the destination airport, so we can choose only DEST variable to represent all these destination related variables.

In [68]:
# delete destination related redundant variables
del_list_dest = ['DEST_AIRPORT_ID', 'DEST_AIRPORT_SEQ_ID', 'DEST_CITY_MARKET_ID', 'DEST_CITY_NAME', 'DEST_STATE_ABR', 'DEST_STATE_FIPS', 'DEST_STATE_NM', 'DEST_WAC']

for col in del_list_dest:
  del df_airlines[col]
 
df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY 
 DEP_DELAY_NEW 
 DEP_DEL15 
 DEP_DELAY_GROUP 
 DEP_TIME_BLK 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ARR_DELAY_GROUP 
 ARR_TIME_BLK 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 DISTANCE_GROUP 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 5.0 
 5.0 
 0.0 
 0.0 
 1000-1059 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 10.0 
 0.0 
 0.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 5 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 16.0 
 16.0 
 1.0 
 1.0 
 1000-1059 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 22.0 
 1.0 
 1.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 5 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 6.0 
 6.0 
 0.0 
 0.0 
 1000-1059 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 42.0 
 1.0 
 2.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 5 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 32.0 
 32.0 
 1.0 
 2.0 
 1000-1059 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 68.0 
 1.0 
 4.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 5 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 23.0 
 23.0 
 1.0 
 1.0 
 1000-1059 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 34.0 
 34.0 
 1.0 
 2.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 5 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN

##### Departure Performance related variables:
1) 
- DEP_DELAY	
- **DEP_DELAY_NEW**
- DEP_DEL15	
- DEP_DELAY_GROUP

DEP_DELAY_NEW changes the early departure which is the negetive value in DEP_DELAY to 0. DEP_DEL15 is indicate if the delay is over 15 mins. DEP_DELAY_GROUP categorize the degree of DEP_DELAY into different groups. So we can use DEP_DELAY_NEW to represent these variables.

2)
- **CRS_DEP_TIME**
- DEP_TIME_BLK

DEP_TIME_BLK is changing CRS_DEP_TIME to hourly intervals.
So we can use CRS_DEP_TIME to represent these variables.

In [70]:
# delete departure performance related redundant variables
del_list_dest_perf = ['DEP_DELAY', 'DEP_DEL15', 'DEP_DELAY_GROUP', 'DEP_TIME_BLK']

for col in del_list_dest_perf:
  del df_airlines[col]

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY_NEW 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY 
 ARR_DELAY_NEW 
 ARR_DEL15 
 ARR_DELAY_GROUP 
 ARR_TIME_BLK 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 DISTANCE_GROUP 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 5.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 10.0 
 0.0 
 0.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 5 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 16.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 22.0 
 1.0 
 1.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 5 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 6.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 42.0 
 1.0 
 2.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 5 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 32.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 68.0 
 1.0 
 4.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 5 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 23.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 34.0 
 34.0 
 1.0 
 2.0 
 1400-1459 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 5 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 


##### Arrival Performance related variables:

1) 
- ARR_DELAY	
- **ARR_DELAY_NEW**
- ARR_DEL15	
- ARR_DELAY_GROUP

ARR_DELAY_NEW changes the early departure which is the negative value in ARR_DELAY to 0. ARR_DEL15 is indicate if the delay is over 15 mins. ARR_DELAY_GROUP categorize the degree of ARR_DELAY into different groups. So we can use ARR_DELAY_NEW to represent these variables.

2)
- **CRS_ARR_TIME**
- ARR_TIME_BLK

ARR_TIME_BLK is changing CRS_ARR_TIME to hourly intervals.
So we can use CRS_ARR_TIME to represent these variables.

In [72]:
# delete arrival performance related redundant variables
del_list_arr_perf = ['ARR_DELAY', 'ARR_DEL15', 'ARR_DELAY_GROUP', 'ARR_TIME_BLK']

for col in del_list_arr_perf:
  del df_airlines[col]

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY_NEW 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY_NEW 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 DISTANCE_GROUP 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 ... 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 5.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 5 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 16.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 5 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 6.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 5 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 32.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 5 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 23.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 34.0 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 5 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 ... 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 N

##### Flight summaries
- **DISTANCE**	
- DISTANCE_GROUP

DISTANCE_GROUP is changing DISTANCE to distance Intervals, every 250 Miles, for Flight Segment. So we can use DISTANCE to represent these variables.

In [74]:
# delete flight summaries related redundant variables
del df_airlines['DISTANCE_GROUP']

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY_NEW 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY_NEW 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DIV4_AIRPORT 
 DIV4_AIRPORT_ID 
 DIV4_AIRPORT_SEQ_ID 
 DIV4_WHEELS_ON 
 DIV4_TOTAL_GTIME 
 DIV4_LONGEST_GTIME 
 DIV4_WHEELS_OFF 
 DIV4_TAIL_NUM 
 DIV5_AIRPORT 
 DIV5_AIRPORT_ID 
 DIV5_AIRPORT_SEQ_ID 
 DIV5_WHEELS_ON 
 DIV5_TOTAL_GTIME 
 DIV5_LONGEST_GTIME 
 DIV5_WHEELS_OFF 
 DIV5_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 5.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 16.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 6.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 32.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 23.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 34.0 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 None 
 None 
 None 
 None 
 None 
 Non

##### Diverted Airport Information
- DIV4_AIRPORT	
- DIV4_AIRPORT_ID	
- DIV4_AIRPORT_SEQ_ID	
- DIV4_WHEELS_ON	
- DIV4_TOTAL_GTIME	
- DIV4_LONGEST_GTIME	
- DIV4_WHEELS_OFF	
- DIV4_TAIL_NUM	
- DIV5_AIRPORT	
- DIV5_AIRPORT_ID	
- DIV5_AIRPORT_SEQ_ID	
- DIV5_WHEELS_ON	
- DIV5_TOTAL_GTIME	
- DIV5_LONGEST_GTIME	
- DIV5_WHEELS_OFF	
- DIV5_TAIL_NUM

DIV4 and DIV5 related variables do not have data, so we will remove them.

In [76]:
# check if there is values from DIV1 to DIV5 related variables
df_airlines.describe()

OP_CARRIER_AIRLINE_ID 
 OP_CARRIER_FL_NUM 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY_NEW 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY_NEW 
 CANCELLED 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DELAY_15 
 
 
 
 
 count 
 161057.000000 
 161057.000000 
 161057.000000 
 156231.000000 
 156231.000000 
 156121.000000 
 156121.000000 
 155957.000000 
 155957.000000 
 161057.000000 
 155957.000000 
 155692.000000 
 161057.000000 
 161057.000000 
 161057.000000 
 155692.000000 
 155692.000000 
 161057.0 
 161057.000000 
 34915.000000 
 34915.000000 
 34915.00000 
 34915.000000 
 34915.000000 
 1053.000000 
 1053.000000 
 1053.000000 
 161057.000000 
 381.000000 
 265.000000 
 265.000000 
 381.000000 
 429.000000 
 4.290000e+02 
 429.000000 
 429.000000 
 429.000000 
 278.000000 
 17.000000 
 1.700000e+01 
 17.000000 
 17.000000 
 17.000000 
 5.000000 
 1.0 
 1.0 
 1.0 
 1.0 
 1.0 
 161057.000000 
 
 
 mean 
 20010.918153 
 2508.798711 
 1422.252768 
 1437.216897 
 14.368870 
 18.062086 
 1465.591503 
 1563.578172 
 6.149086 
 1584.805361 
 1566.326064 
 14.163483 
 0.030946 
 0.002366 
 124.434784 
 120.037144 
 95.836177 
 1.0 
 662.231340 
 19.903050 
 6.769039 
 12.16838 
 0.017557 
 19.076672 
 1399.992403 
 36.226021 
 35.970560 
 0.005116 
 0.695538 
 347.101887 
 232.633962 
 47.908136 
 12499.242424 
 1.249927e+06 
 1464.046620 
 27.405594 
 22.242424 
 1474.884892 
 12674.176471 
 1.267421e+06 
 1341.411765 
 11.941176 
 10.235294 
 1392.200000 
 10397.0 
 1039705.0 
 51.0 
 4.0 
 4.0 
 0.227143 
 
 
 std 
 314.626497 
 1746.419264 
 453.970474 
 463.857461 
 35.596949 
 9.087123 
 465.871793 
 488.380807 
 4.818137 
 467.098093 
 494.175085 
 36.196183 
 0.173171 
 0.048580 
 64.537094 
 63.288804 
 61.754708 
 0.0 
 474.807024 
 38.098095 
 29.870353 
 25.07071 
 0.805233 
 36.269235 
 482.825093 
 28.256333 
 27.885904 
 0.163356 
 0.460784 
 204.471624 
 206.620475 
 124.824801 
 1657.100266 
 1.657094e+05 
 564.358918 
 25.880631 
 24.029032 
 543.504354 
 1805.210349 
 1.805201e+05 
 617.786073 
 9.222734 
 7.198345 
 586.312374 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0.418987 
 
 
 min 
 19393.000000 
 3.000000 
 500.000000 
 1.000000 
 0.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 1.000000 
 0.000000 
 0.000000 
 0.000000 
 37.000000 
 27.000000 
 14.000000 
 1.0 
 67.000000 
 0.000000 
 0.000000 
 0.00000 
 0.000000 
 0.000000 
 25.000000 
 1.000000 
 1.000000 
 0.000000 
 0.000000 
 91.000000 
 -6.000000 
 0.000000 
 10135.000000 
 1.013503e+06 
 6.000000 
 2.000000 
 2.000000 
 11.000000 
 10397.000000 
 1.039705e+06 
 50.000000 
 3.000000 
 3.000000 
 824.000000 
 10397.0 
 1039705.0 
 51.0 
 4.0 
 4.0 
 0.000000 
 
 
 25% 
 19790.000000 
 1171.000000 
 1025.000000 
 1029.000000 
 0.000000 
 13.000000 
 1046.000000 
 1155.000000 
 4.000000 
 1206.000000 
 1159.000000 
 0.000000 
 0.000000 
 0.000000 
 81.000000 
 78.000000 
 55.000000 
 1.0 
 332.000000 
 0.000000 
 0.000000 
 0.00000 
 0.000000 
 0.000000 
 953.000000 
 19.000000 
 19.000000 
 0.000000 
 0.000000 
 231.000000 
 127.000000 
 0.000000 
 10821.000000 
 1.082103e+06 
 1045.000000 
 10.000000 
 9.000000 
 1121.250000 
 10397.000000 
 1.039705e+06 
 1025.000000 
 7.000000 
 7.000000 
 1018.000000 
 10397.0 
 1039705.0 
 51.0 
 4.0 
 4.0 
 0.000000 
 
 
 50% 
 19805.000000 
 1990.000000 
 1425.000000 
 1444.00000

In [77]:
# delete diverted airport information related redundant variables
del_list_div_info = ['DIV4_AIRPORT', 'DIV4_AIRPORT_ID', 'DIV4_AIRPORT_SEQ_ID', 'DIV4_WHEELS_ON', 'DIV4_TOTAL_GTIME', 'DIV4_LONGEST_GTIME', 'DIV4_WHEELS_OFF', 'DIV4_TAIL_NUM', 'DIV5_AIRPORT', 'DIV5_AIRPORT_ID', 'DIV5_AIRPORT_SEQ_ID', 'DIV5_WHEELS_ON', 'DIV5_TOTAL_GTIME', 'DIV5_LONGEST_GTIME', 'DIV5_WHEELS_OFF', 'DIV5_TAIL_NUM']

for col in del_list_div_info:
  del df_airlines[col]

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 DEP_DELAY_NEW 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 ARR_DELAY_NEW 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 5.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 10.0 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 16.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 22.0 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 6.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 42.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 32.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 68.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 23.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 34.0 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0

#### 2-2. Check the highly correlated variables from correlation matrix

In [79]:
##Correlation Matrix
air_corr = df_airlines.corr()
fig, ax = plt.subplots(figsize=(20, 20))
mask = np.zeros_like(air_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sn.diverging_palette(240, 10, as_cmap=True)
sn.heatmap(air_corr, mask=mask, cmap=cmap, center=0, linewidths=.5)
plt.title("Correlations between features.")
display(plt.show())

#### 2-3. Remove the variables which is related to the outcome "DELAY_15"
- DEP_DELAY_NEW
- ARR_DELAY_NEW

In [81]:
# delete outcome related variables
del_list_outcome_related = ['DEP_DELAY_NEW', 'ARR_DELAY_NEW']

for col in del_list_outcome_related:
  del df_airlines[col]

df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DELAY_15 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0

In [82]:
df_airlines.shape

Out[52]: (161057, 62)

### 3. Adjust the Departure time
The actually departure time is **DEP_TIME** variable, we need to adjust the departure time of canceled, and use this **ADJ_DEP_TIME** to align with the weather data. The divert flights affect the arrival time, but not the departure time.

#### 3-1. First set the value of ADJ_DEP_TIME as DEP_TIME

In [85]:
# Assign the value of ADJ_DEP_TIME as DEP_TIME
df_airlines['ADJ_DEP_TIME'] = df_airlines.DEP_TIME
df_airlines.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DELAY_15 
 ADJ_DEP_TIME 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 0.0 
 None 
 0.0 
 160.0 
 165.0 
 144.0 
 1.0 
 1120.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 1040.0 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 0.0 
 None 
 0.0 
 160.0 
 166.0 
 152.0 
 1.0 
 1120.0 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1051.0 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 154.0 
 1.0 
 1120.0 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 1041.0 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 0.0 
 None 
 0.0 
 160.0 
 196.0 
 169.0 
 1.0 
 1120.0 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1107.0 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 0.0 
 None 
 0.0 
 160.0 
 171.0 
 145.0 
 1.0 
 1120.0 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1058.0

#### 3-2. Set NaN value in ADJ_DEP_TIME variable in canceled flight.
Canceled flights should not have a departure time.

In [87]:
# Check the departure time of the canceled flight
df_airlines.loc[df_airlines['CANCELLED'] == 1]
# Most the DEP_TIME value is NaN, but some had value, we need to remove it as NaN

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DELAY_15 
 ADJ_DEP_TIME 
 
 
 
 
 35 
 2015-01-05 
 AA 
 19805 
 AA 
 N467AA 
 1384 
 ORD 
 MSP 
 2055 
 25.0 
 NaN 
 NaN 
 NaN 
 NaN 
 2220 
 NaN 
 1.0 
 B 
 0.0 
 85.0 
 NaN 
 NaN 
 1.0 
 334.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 25.0 
 116.0 
 116.0 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 25.0 
 
 
 64 
 2015-01-03 
 AA 
 19805 
 AA 
 N564AA 
 1331 
 ORD 
 STL 
 1955 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 2100 
 NaN 
 1.0 
 A 
 0.0 
 65.0 
 NaN 
 NaN 
 1.0 
 258.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 NaN 
 
 
 157 
 2015-01-26 
 AA 
 19805 
 AA 
 N3CCAA 
 1426 
 ORD 
 BOS 
 1835 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 2149 
 NaN 
 1.0 
 B 
 0.0 
 134.0 
 NaN 
 NaN 
 1.0 
 867.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 NaN 
 
 
 158 
 2015-01-27 
 AA 
 19805 
 AA 
 None 
 1426 
 ORD 
 BOS 
 1835 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 2149 
 NaN 
 1.0 
 B 
 0.0 
 134.0 
 NaN 
 NaN 
 1.0 
 867.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 NaN 
 
 
 364 
 2015-01-27 
 AA 
 19805 
 AA 
 None 
 1496 
 ORD 
 BOS 
 800 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 1110 
 NaN 
 1.0 
 B 
 0.0 
 130.0 
 NaN 
 NaN 
 1.0 
 867.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 NaN 
 
 
 365 
 2015-01-28 
 AA 
 19805 
 AA 
 N3LMAA 
 1496 
 ORD 
 BOS 
 800 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 1110 
 NaN 
 1.0 
 B 
 0.0 
 130.0 
 NaN 
 NaN 
 1.0 
 867.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 NaN 
 
 
 369 
 2015-01-01 
 AA 
 19805 
 AA 
 N005AA 
 1080 
 ORD 
 EGE 
 1145 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 1335 
 NaN 
 1.0 
 A 
 0.0 
 170.0 
 NaN 
 NaN 
 1.0 
 1007.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 NaN 
 
 
 373 
 2015-01-05 
 AA 
 19805 
 AA 
 N015AA 
 1080 
 ORD 
 EGE 
 1145 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 1335 
 

In [88]:
# Set NaN value in ADJ_DEP_TIME variable in canceled flight.
df_airlines.loc[df_airlines['CANCELLED'] == 1, 'ADJ_DEP_TIME'] = np.nan

# Check if the ADJ_DEP_TIME variable in canceled flight is NaN
df_airlines.loc[df_airlines['CANCELLED'] == 1].head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 FLIGHTS 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DELAY_15 
 ADJ_DEP_TIME 
 
 
 
 
 35 
 2015-01-05 
 AA 
 19805 
 AA 
 N467AA 
 1384 
 ORD 
 MSP 
 2055 
 25.0 
 NaN 
 NaN 
 NaN 
 NaN 
 2220 
 NaN 
 1.0 
 B 
 0.0 
 85.0 
 NaN 
 NaN 
 1.0 
 334.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 25.0 
 116.0 
 116.0 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 NaN 
 
 
 64 
 2015-01-03 
 AA 
 19805 
 AA 
 N564AA 
 1331 
 ORD 
 STL 
 1955 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 2100 
 NaN 
 1.0 
 A 
 0.0 
 65.0 
 NaN 
 NaN 
 1.0 
 258.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 NaN 
 
 
 157 
 2015-01-26 
 AA 
 19805 
 AA 
 N3CCAA 
 1426 
 ORD 
 BOS 
 1835 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 2149 
 NaN 
 1.0 
 B 
 0.0 
 134.0 
 NaN 
 NaN 
 1.0 
 867.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 NaN 
 
 
 158 
 2015-01-27 
 AA 
 19805 
 AA 
 None 
 1426 
 ORD 
 BOS 
 1835 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 2149 
 NaN 
 1.0 
 B 
 0.0 
 134.0 
 NaN 
 NaN 
 1.0 
 867.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 NaN 
 
 
 364 
 2015-01-27 
 AA 
 19805 
 AA 
 None 
 1496 
 ORD 
 BOS 
 800 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 1110 
 NaN 
 1.0 
 B 
 0.0 
 130.0 
 NaN 
 NaN 
 1.0 
 867.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 NaN

### 4. Normalization of Data

##### Standardize continuous variables
- CRS_ELAPSED_TIME	
- ACTUAL_ELAPSED_TIME	
- AIR_TIME 
- DISTANCE	
- CARRIER_DELAY	
- WEATHER_DELAY	
- NAS_DELAY	
- SECURITY_DELAY	
- LATE_AIRCRAFT_DELAY 
- DIV_ACTUAL_ELAPSED_TIME 
- DIV_ARR_DELAY	
- DIV_DISTANCE

Some of the variables in airlines is date, str or categorical data, so we just select continuous variables to standardize.

In [91]:
# import Standardize package
from sklearn.preprocessing import StandardScaler

In [92]:
# Get continuous variables
X_continuous =df_airlines[['CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'DIV_ACTUAL_ELAPSED_TIME', 'DIV_ARR_DELAY', 'DIV_DISTANCE']]
X_continuous.head()

CRS_ELAPSED_TIME 
 ACTUAL_ELAPSED_TIME 
 AIR_TIME 
 DISTANCE 
 CARRIER_DELAY 
 WEATHER_DELAY 
 NAS_DELAY 
 SECURITY_DELAY 
 LATE_AIRCRAFT_DELAY 
 DIV_ACTUAL_ELAPSED_TIME 
 DIV_ARR_DELAY 
 DIV_DISTANCE 
 
 
 
 
 0 
 160.0 
 165.0 
 144.0 
 1120.0 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 
 
 1 
 160.0 
 166.0 
 152.0 
 1120.0 
 16.0 
 0.0 
 6.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 
 
 2 
 160.0 
 196.0 
 154.0 
 1120.0 
 6.0 
 0.0 
 36.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 
 
 3 
 160.0 
 196.0 
 169.0 
 1120.0 
 0.0 
 0.0 
 68.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN 
 
 
 4 
 160.0 
 171.0 
 145.0 
 1120.0 
 23.0 
 0.0 
 11.0 
 0.0 
 0.0 
 NaN 
 NaN 
 NaN

In [93]:
# Standardizing the continuous variables
X = StandardScaler().fit_transform(X_continuous)
X

Out[58]: array([[ 0.55108348, 0.71044162, 0.77992398, ..., nan,
 nan, nan],
 [ 0.55108348, 0.72624226, 0.90946918, ..., nan,
 nan, nan],
 [ 0.55108348, 1.20026127, 0.94185548, ..., nan,
 nan, nan],
 ...,
 [-0.61104249, -0.58521035, -0.43456221, ..., nan,
 nan, nan],
 [-0.53356742, -0.68001415, -0.54791426, ..., nan,
 nan, nan],
 [-0.53356742, -0.60101098, -0.46694851, ..., nan,
 nan, nan]])

### 5. Dimensionality reduction

#### 5-1. Use PCA for continuous variables

In [96]:
# import PCA package
from sklearn.decomposition import PCA

In [97]:
# fill in all na value to 0 because PCA can't work on NaN value
X[np.isnan(X)] = 0

In [98]:
# Use PCA, reduce the feature vector to 2D
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
df_principal = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
df_principal.head()

principal component 1 
 principal component 2 
 
 
 
 
 0 
 1.502625 
 0.007499 
 
 
 1 
 1.575095 
 0.268880 
 
 
 2 
 1.835153 
 -0.554518 
 
 
 3 
 1.966342 
 -1.303872 
 
 
 4 
 1.559364 
 0.293698

#### 5-2. Combine the airlines together (PCA continuous variables + non-continuous variables)

In [100]:
# get non-continuous variables
df_airlines_non_continuous = df_airlines.drop(['CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'DIV_ACTUAL_ELAPSED_TIME', 'DIV_ARR_DELAY', 'DIV_DISTANCE'], axis=1)
df_airlines_non_continuous.shape

Out[62]: (161057, 51)

In [101]:
# concat airlines PCA continuous and non-continuous variables
df_airlines_pca = pd.concat([df_airlines_non_continuous, df_principal], axis=1)
df_airlines_pca.shape

Out[63]: (161057, 53)

In [102]:
df_airlines_pca.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 FLIGHTS 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DELAY_15 
 ADJ_DEP_TIME 
 principal component 1 
 principal component 2 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 1040.0 
 1.502625 
 0.007499 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1051.0 
 1.575095 
 0.268880 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 1041.0 
 1.835153 
 -0.554518 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1107.0 
 1.966342 
 -1.303872 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1058.0 
 1.559364 
 0.293698

In [103]:
print(df_airlines_pca.iloc[0,8])
print(type(df_airlines_pca.iloc[0, 8]))
type(int(df_airlines_pca.iloc[0, -3]))

1035
<class 'numpy.int32'>
Out[65]: int

###6. Feature Reduction for Weather Data

#### 6-1. Check weather data's location and time
- Location: CALL_SIGN -> ORIGIN
- Time: DATE +2hr -> DEP_HOUR, FL_DATE

In [106]:
weather.select('CALL_SIGN').distinct().collect()

Out[66]: [Row(CALL_SIGN='KBHB '),
 Row(CALL_SIGN='KTZR '),
 Row(CALL_SIGN='KCEY '),
 Row(CALL_SIGN='KPRX '),
 Row(CALL_SIGN='KGGP '),
 Row(CALL_SIGN='KLBE '),
 Row(CALL_SIGN='KMGJ '),
 Row(CALL_SIGN='KBYI '),
 Row(CALL_SIGN='TJNR '),
 Row(CALL_SIGN='KFAR '),
 Row(CALL_SIGN='KRQB '),
 Row(CALL_SIGN='KMGR '),
 Row(CALL_SIGN='KLAN '),
 Row(CALL_SIGN='KBQK '),
 Row(CALL_SIGN='KAVK '),
 Row(CALL_SIGN='KPDT '),
 Row(CALL_SIGN='PAFM '),
 Row(CALL_SIGN='KGYB '),
 Row(CALL_SIGN='KGMU '),
 Row(CALL_SIGN='PALJ '),
 Row(CALL_SIGN='KMRN '),
 Row(CALL_SIGN='KPTW '),
 Row(CALL_SIGN='KDMN '),
 Row(CALL_SIGN='KBXA '),
 Row(CALL_SIGN='KGCC '),
 Row(CALL_SIGN='KHOE '),
 Row(CALL_SIGN='KPUJ '),
 Row(CALL_SIGN='PADE '),
 Row(CALL_SIGN='K40B '),
 Row(CALL_SIGN='KLKU '),
 Row(CALL_SIGN='KSBA '),
 Row(CALL_SIGN='KOAJ '),
 Row(CALL_SIGN='KHST '),
 Row(CALL_SIGN='KCKP '),
 Row(CALL_SIGN='KMMU '),
 Row(CALL_SIGN='KCIR '),
 Row(CALL_SIGN='KRSW '),
 Row(CALL_SIGN='KROA '),
 Row(CALL_SIGN='KFYG '),
 Row(CALL_SIGN='PABT '),
 Row(CALL_SIGN='KHYW '),
 Row(CALL_SIGN='KIKV '),
 Row(CALL_SIGN='KARA '),
 Row(CALL_SIGN='KBEH '),
 Row(CALL_SIGN='KALM '),
 Row(CALL_SIGN='KPNE '),
 Row(CALL_SIGN='K14Y '),
 Row(CALL_SIGN='KCNK '),
 Row(CALL_SIGN='KDYL '),
 Row(CALL_SIGN='KSAZ '),
 Row(CALL_SIGN='KFKA '),
 Row(CALL_SIGN='KTOI '),
 Row(CALL_SIGN='KMSO '),
 Row(CALL_SIGN='KPNM '),
 Row(CALL_SIGN='KEHO '),
 Row(CALL_SIGN='K8A0 '),
 Row(CALL_SIGN='KCMR '),
 Row(CALL_SIGN='KAIK '),
 Row(CALL_SIGN='KGRY '),
 Row(CALL_SIGN='KSSC '),
 Row(CALL_SIGN='KGIF '),
 Row(CALL_SIGN='KATY '),
 Row(CALL_SIGN='KELN '),
 Row(CALL_SIGN='KNFG '),
 Row(CALL_SIGN='KHAE '),
 Row(CALL_SIGN='PAMC '),
 Row(CALL_SIGN='KHDC '),
 Row(CALL_SIGN='KCOT '),
 Row(CALL_SIGN='KMHV '),
 Row(CALL_SIGN='KRGK '),
 Row(CALL_SIGN='KTRI '),
 Row(CALL_SIGN='KGCD '),
 Row(CALL_SIGN='KDEH '),
 Row(CALL_SIGN='KNFW '),
 Row(CALL_SIGN='KWHP '),
 Row(CALL_SIGN='KTRL '),
 Row(CALL_SIGN='KSIY '),
 Row(CALL_SIGN='KPRN '),
 Row(CALL_SIGN='KZZV '),
 Row(CALL_SIGN='K65S '),
 Row(CALL_SIGN='KSMX '),
 Row(CALL_SIGN='KDDH '),
 Row(CALL_SIGN='PAOR '),
 Row(CALL_SIGN='KMPZ '),
 Row(CALL_SIGN='KCPW '),
 Row(CALL_SIGN='KM40 '),
 Row(CALL_SIGN='KROC '),
 Row(CALL_SIGN='KSRR '),
 Row(CALL_SIGN='KPWM '),
 Row(CALL_SIGN='KSFM '),
 Row(CALL_SIGN='KMKL '),
 Row(CALL_SIGN='KRAS '),
 Row(CALL_SIGN='KVRB '),
 Row(CALL_SIGN='K1A6 '),
 Row(CALL_SIGN='KUCY '),
 Row(CALL_SIGN='KLWS '),
 Row(CALL_SIGN='TJIG '),
 Row(CALL_SIGN='KCKV '),
 Row(CALL_SIGN='KBUY '),
 Row(CALL_SIGN='KUGN '),
 Row(CALL_SIGN='PAGL '),
 Row(CALL_SIGN='K12N '),
 Row(CALL_SIGN='KD95 '),
 Row(CALL_SIGN='KMLE '),
 Row(CALL_SIGN='KECP '),
 Row(CALL_SIGN='KBLV '),
 Row(CALL_SIGN='K05U '),
 Row(CALL_SIGN='KDLN '),
 Row(CALL_SIGN='KMLT '),
 Row(CALL_SIGN='K19A '),
 Row(CALL_SIGN='K2G4 '),
 Row(CALL_SIGN='KH78 '),
 Row(CALL_SIGN='PAAK '),
 Row(CALL_SIGN='KTTD '),
 Row(CALL_SIGN='KLBF '),
 Row(CALL_SIGN='KP58 '),
 Row(CALL_SIGN='KDPA '),
 Row(CALL_SIGN='KWLD '),
 Row(CALL_SIGN='KMNI '),
 Row(CALL_SIGN='KSWW '),
 Row(CALL_SIGN='KW22 '),
 Row(CALL_SIGN='KDBQ '),
 Row(CALL_SIGN='KHRI '),
 Row(CALL_SIGN='KPAE '),
 Row(CALL_SIGN='KMTJ '),
 Row(CALL_SIGN='KCRE '),
 Row(CALL_SIGN='KHLX '),
 Row(CALL_SIGN='KLAS '),
 Row(CALL_SIGN='KCRS '),
 Row(CALL_SIGN='KDTN '),
 Row(CALL_SIGN='KBLF '),
 Row(CALL_SIGN='KDFI '),
 Row(CALL_SIGN='KASG '),
 Row(CALL_SIGN='KGDV '),
 Row(CALL_SIGN='KBKE '),
 Row(CALL_SIGN='KSQL '),
 Row(CALL_SIGN='KAQP '),
 Row(CALL_SIGN='KEMT '),
 Row(CALL_SIGN='KSUX '),
 Row(CALL_SIGN='PATO '),
 Row(CALL_SIGN='KMPO '),
 Row(CALL_SIGN='KRYT '),
 Row(CALL_SIGN='PASD '),
 Row(CALL_SIGN='PALH '),
 Row(CALL_SIGN='KDAL '),
 Row(CALL_SIGN='KDGW '),
 Row(CALL_SIGN='KPVW '),
 Row(CALL_SIGN='KALN '),
 Row(CALL_SIGN='KW99 '),
 Row(CALL_SIGN='KCHD '),
 Row(CALL_SIGN='KSDL '),
 Row(CALL_SIGN='KGEY '),
 Row(CALL_SIGN='KIDP '),
 Row(CALL_SIGN='KAMW '),
 Row(CALL_SIGN='KMQB '),
 Row(CALL_SIGN='KPQL '),
 Row(CALL_SIGN='KCKC '),
 Row(CALL_SIGN='KENW '),
 Row(CALL_SIGN='KGUY '),
 Row(CALL_SIGN='

In [107]:
# Select CALL_SIGN near ORD, ATL
# ATL closest station is KATL
# ORD closest station is KORD
weather_loc = weather.where(f.col("CALL_SIGN").isin({"KATL ", "KORD "}))
weather_loc.count()

Out[67]: 5863

In [108]:
# Change the weather_loc to dataframe
df_weather = weather_loc.toPandas()
df_weather.shape

Out[68]: (5863, 177)

In [109]:
df_weather.describe()

SOURCE 
 LATITUDE 
 LONGITUDE 
 ELEVATION 
 
 
 
 
 count 
 5863.000000 
 5863.000000 
 5863.000000 
 5863.000000 
 
 
 mean 
 6.936722 
 37.924548 
 -86.234452 
 253.380761 
 
 
 std 
 0.243483 
 4.181307 
 1.745423 
 52.985513 
 
 
 min 
 6.000000 
 33.630100 
 -87.933600 
 201.800000 
 
 
 25% 
 7.000000 
 33.630100 
 -87.933600 
 201.800000 
 
 
 50% 
 7.000000 
 41.995000 
 -87.933600 
 201.800000 
 
 
 75% 
 7.000000 
 41.995000 
 -84.441800 
 307.800000 
 
 
 max 
 7.000000 
 41.995000 
 -84.441800 
 307.800000

In [110]:
df_weather.head()

STATION 
 DATE 
 SOURCE 
 LATITUDE 
 LONGITUDE 
 ELEVATION 
 NAME 
 REPORT_TYPE 
 CALL_SIGN 
 QUALITY_CONTROL 
 WND 
 CIG 
 VIS 
 TMP 
 DEW 
 SLP 
 AW1 
 GA1 
 GA2 
 GA3 
 GA4 
 GE1 
 GF1 
 KA1 
 KA2 
 MA1 
 MD1 
 MW1 
 MW2 
 OC1 
 OD1 
 OD2 
 REM 
 EQD 
 AW2 
 AX4 
 GD1 
 AW5 
 GN1 
 AJ1 
 ... 
 GA6 
 KG1 
 AU3 
 AT7 
 KD1 
 GL1 
 IA1 
 GG2 
 OD3 
 UG1 
 CB1 
 AI6 
 CI1 
 CV2 
 AZ2 
 AD1 
 AH1 
 WD1 
 AA4 
 KC1 
 IA2 
 CF3 
 AI2 
 AT1 
 GD4 
 AX3 
 AH4 
 KB3 
 CU1 
 CN4 
 AT2 
 CG1 
 CF1 
 GG1 
 MV2 
 CW1 
 GG4 
 AB1 
 AH5 
 CN3 
 
 
 
 
 0 
 72530094846 
 2015-01-01 00:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0057,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10234,5 
 
 02,5,+07620,5,99,9 
 
 
 
 9,AGL ,+99999,+99999 
 02,99,5,99,9,99,9,07620,1,99,9,99,9 
 
 
 10220,5,09974,5 
 
 
 
 0118,5 
 
 
 MET10412/31/14 18:51:03 METAR KORD 010051Z 240... 
 D01 0ADE726 
 
 
 1,99,1,+07620,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 72530094846 
 2015-01-01 01:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 230,5,N,0077,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0167,5 
 10235,5 
 
 02,5,+03048,5,99,9 
 02,5,+07620,5,99,9 
 
 
 9,AGL ,+99999,+99999 
 02,99,5,99,9,99,9,03048,1,99,9,99,9 
 
 
 10220,5,09974,5 
 
 
 
 0113,5 
 4,99,0139,1,230 
 
 MET12912/31/14 19:51:03 METAR KORD 010151Z 230... 
 D01 0ADE726 
 
 
 1,99,1,+03048,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 72530094846 
 2015-01-01 02:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0072,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10232,5 
 
 02,5,+07620,5,99,9 
 
 
 
 9,AGL ,+99999,+99999 
 02,99,5,99,9,99,9,07620,1,99,9,99,9 
 
 
 10220,5,09974,5 
 6,9,009,9,+999,9 
 
 
 0139,5 
 
 
 MET12812/31/14 20:51:03 METAR KORD 010251Z 240... 
 D01 0ADE726 
 
 
 1,99,1,+07620,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 72530094846 
 2015-01-01 03:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0067,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10229,5 
 
 00,5,+99999,9,99,9 
 
 
 
 
 00,99,1,99,9,99,9,99999,9,99,9,99,9 
 
 
 10217,5,09971,5 
 
 
 
 0103,5 
 4,99,0133,1,240 
 
 MET11912/31/14 21:51:03 METAR KORD 010351Z 240... 
 
 
 
 0,99,1,+99999,9,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4 
 72530094846 
 2015-01-01 04:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0088,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10223,5 
 
 00,5,+99999,9,99,9 
 
 
 
 
 00,99,1,99,9,99,9,99999,9,99,9,99,9 
 
 
 10210,5,09964,5 
 
 
 
 0124,5 
 
 
 MET10112/31/14 22:51:03 METAR KORD 010451Z 240... 
 
 
 
 0,99,1,+99999,9,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 5 rows × 177 columns

In [111]:
# Add ORIGIN column in weather to represent the location
df_weather.loc[df_weather['CALL_SIGN'] == 'KATL ', 'ORIGIN'] = 'ATL' 
df_weather.loc[df_weather['CALL_SIGN'] == 'KORD ', 'ORIGIN'] = 'ORD' 

In [112]:
# Add 2 hours in weather data because we want the predict the delay 2 hours before departure. Add DEP_HOUR and DEP_DATE columns in weather to represent the departure time
df_weather['DEP_HOUR'] = df_weather.DATE.map(lambda x: (x + datetime.timedelta(hours=2)).hour)
df_weather['FL_DATE'] = df_weather.DATE.map(lambda x: (x + datetime.timedelta(hours=2)).date())
df_weather.head()

STATION 
 DATE 
 SOURCE 
 LATITUDE 
 LONGITUDE 
 ELEVATION 
 NAME 
 REPORT_TYPE 
 CALL_SIGN 
 QUALITY_CONTROL 
 WND 
 CIG 
 VIS 
 TMP 
 DEW 
 SLP 
 AW1 
 GA1 
 GA2 
 GA3 
 GA4 
 GE1 
 GF1 
 KA1 
 KA2 
 MA1 
 MD1 
 MW1 
 MW2 
 OC1 
 OD1 
 OD2 
 REM 
 EQD 
 AW2 
 AX4 
 GD1 
 AW5 
 GN1 
 AJ1 
 ... 
 AT7 
 KD1 
 GL1 
 IA1 
 GG2 
 OD3 
 UG1 
 CB1 
 AI6 
 CI1 
 CV2 
 AZ2 
 AD1 
 AH1 
 WD1 
 AA4 
 KC1 
 IA2 
 CF3 
 AI2 
 AT1 
 GD4 
 AX3 
 AH4 
 KB3 
 CU1 
 CN4 
 AT2 
 CG1 
 CF1 
 GG1 
 MV2 
 CW1 
 GG4 
 AB1 
 AH5 
 CN3 
 ORIGIN 
 DEP_HOUR 
 FL_DATE 
 
 
 
 
 0 
 72530094846 
 2015-01-01 00:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0057,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10234,5 
 
 02,5,+07620,5,99,9 
 
 
 
 9,AGL ,+99999,+99999 
 02,99,5,99,9,99,9,07620,1,99,9,99,9 
 
 
 10220,5,09974,5 
 
 
 
 0118,5 
 
 
 MET10412/31/14 18:51:03 METAR KORD 010051Z 240... 
 D01 0ADE726 
 
 
 1,99,1,+07620,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ORD 
 2 
 2015-01-01 
 
 
 1 
 72530094846 
 2015-01-01 01:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 230,5,N,0077,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0167,5 
 10235,5 
 
 02,5,+03048,5,99,9 
 02,5,+07620,5,99,9 
 
 
 9,AGL ,+99999,+99999 
 02,99,5,99,9,99,9,03048,1,99,9,99,9 
 
 
 10220,5,09974,5 
 
 
 
 0113,5 
 4,99,0139,1,230 
 
 MET12912/31/14 19:51:03 METAR KORD 010151Z 230... 
 D01 0ADE726 
 
 
 1,99,1,+03048,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ORD 
 3 
 2015-01-01 
 
 
 2 
 72530094846 
 2015-01-01 02:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0072,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10232,5 
 
 02,5,+07620,5,99,9 
 
 
 
 9,AGL ,+99999,+99999 
 02,99,5,99,9,99,9,07620,1,99,9,99,9 
 
 
 10220,5,09974,5 
 6,9,009,9,+999,9 
 
 
 0139,5 
 
 
 MET12812/31/14 20:51:03 METAR KORD 010251Z 240... 
 D01 0ADE726 
 
 
 1,99,1,+07620,5,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ORD 
 4 
 2015-01-01 
 
 
 3 
 72530094846 
 2015-01-01 03:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0067,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10229,5 
 
 00,5,+99999,9,99,9 
 
 
 
 
 00,99,1,99,9,99,9,99999,9,99,9,99,9 
 
 
 10217,5,09971,5 
 
 
 
 0103,5 
 4,99,0133,1,240 
 
 MET11912/31/14 21:51:03 METAR KORD 010351Z 240... 
 
 
 
 0,99,1,+99999,9,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ORD 
 5 
 2015-01-01 
 
 
 4 
 72530094846 
 2015-01-01 04:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 V030 
 240,5,N,0088,5 
 22000,5,9,N 
 016093,5,N,5 
 -0078,5 
 -0172,5 
 10223,5 
 
 00,5,+99999,9,99,9 
 
 
 
 
 00,99,1,99,9,99,9,99999,9,99,9,99,9 
 
 
 10210,5,09964,5 
 
 
 
 0124,5 
 
 
 MET10112/31/14 22:51:03 METAR KORD 010451Z 240... 
 
 
 
 0,99,1,+99999,9,9 
 
 
 
 ... 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ORD 
 6 
 2015-01-01 
 
 
 
 5 rows × 180 columns

In [113]:
for col in df_weather.columns:
  print(col, df_weather[col].nunique())

STATION 2
DATE 5849
SOURCE 2
LATITUDE 2
LONGITUDE 2
ELEVATION 2
NAME 2
REPORT_TYPE 4
CALL_SIGN 2
QUALITY_CONTROL 1
WND 636
CIG 98
VIS 28
TMP 122
DEW 116
SLP 384
AW1 20
GA1 329
GA2 231
GA3 120
GA4 1
GE1 2
GF1 220
KA1 194
KA2 185
MA1 213
MD1 237
MW1 2
MW2 1
OC1 26
OD1 125
OD2 1
REM 5863
EQD 170
AW2 27
AX4 2
GD1 343
AW5 2
GN1 1
AJ1 37
AW3 11
MK1 7
KA4 40
GG3 1
AN1 30
RH1 45
AU5 1
HL1 1
OB1 1
AT8 1
AW7 1
AZ1 1
CH1 1
RH3 58
GK1 1
IB1 1
AX1 5
CT1 1
AK1 4
CN2 1
OE1 165
MW5 1
AO1 1
KA3 45
AA3 31
CR1 1
CF2 1
KB2 7
GM1 1
AT5 5
AY2 1
MW6 1
MG1 178
AH6 6
AU2 17
GD2 221
AW4 4
MF1 164
AA1 103
AH2 6
AH3 6
OE3 106
AT6 5
AL2 1
AL3 1
AX5 1
IB2 1
AI3 6
CV3 1
WA1 5
GH1 1
KF1 1
CU2 1
CT3 1
SA1 1
AU1 32
KD2 3
AI5 6
GO1 1
GD3 120
CG3 1
AI1 6
AL1 1
AW6 1
MW4 1
AX6 1
CV1 1
ME1 1
KC2 7
CN1 1
UA1 1
GD5 1
UG2 1
AT3 13
AT4 7
GJ1 1
MV1 1
GA5 1
CT2 1
CG2 1
ED1 3
AE1 7
CO1 1
KE1 7
KB1 7
AI4 6
MW3 1
KG2 63
AA2 82
AX2 10
AY1 1
RH2 68
OE2 146
CU3 1
MH1 7
AM1 6
AU4 2
GA6 1
KG1 59
AU3 6
AT7 4
KD1 7
GL1 1
IA1 1
GG2 1
OD3 1
UG1 1
CB1 1
AI6 6
CI1 1
CV2 1
AZ2 1
AD1 7
AH1 6
WD1 1
AA4 1
KC1 7
IA2 1
CF3 1
AI2 6
AT1 9
GD4 19
AX3 5
AH4 6
KB3 7
CU1 1
CN4 1
AT2 13
CG1 1
CF1 1
GG1 1
MV2 1
CW1 1
GG4 1
AB1 7
AH5 6
CN3 1
ORIGIN 2
DEP_HOUR 24
FL_DATE 91

In [114]:
#delete all columns that are blank
cols_to_delete = ['GA4','MW2','OD2','GN1','GG3','AU5','HL1','OB1','AT8','AW7','AZ1','CH1','GK1','IB1','CT1','CN2','MW5','AO1','CR1','CF2','GM1','AY2','MW6',
                 'AL2','AL3','AX5','IB2','CV3','GH1','KF1','CU2','CT3','SA1','GO1','CG3','AL1','AW6','MW4','AX6','CV1','ME1','CN1','UA1','GD5','UG2','GJ1','MV1',
                 'GA5','CT2','CG2','CO1','MW3','AY1','CU3','GA6','GL1','IA1','GG2','OD3','UG1','CB1','CI1','CV2','AZ2','WD1','AA4','IA2','CF3','CU1','CN4','CG1',
                 'CF1','GG1','MV2','CW1','GG4','CN3','QUALITY_CONTROL']
for col in cols_to_delete:
  del df_weather[col]

In [115]:
for col in df_weather.columns:
  print(col, df_weather[col].nunique())

STATION 2
DATE 5849
SOURCE 2
LATITUDE 2
LONGITUDE 2
ELEVATION 2
NAME 2
REPORT_TYPE 4
CALL_SIGN 2
WND 636
CIG 98
VIS 28
TMP 122
DEW 116
SLP 384
AW1 20
GA1 329
GA2 231
GA3 120
GE1 2
GF1 220
KA1 194
KA2 185
MA1 213
MD1 237
MW1 2
OC1 26
OD1 125
REM 5863
EQD 170
AW2 27
AX4 2
GD1 343
AW5 2
AJ1 37
AW3 11
MK1 7
KA4 40
AN1 30
RH1 45
RH3 58
AX1 5
AK1 4
OE1 165
KA3 45
AA3 31
KB2 7
AT5 5
MG1 178
AH6 6
AU2 17
GD2 221
AW4 4
MF1 164
AA1 103
AH2 6
AH3 6
OE3 106
AT6 5
AI3 6
WA1 5
AU1 32
KD2 3
AI5 6
GD3 120
AI1 6
KC2 7
AT3 13
AT4 7
ED1 3
AE1 7
KE1 7
KB1 7
AI4 6
KG2 63
AA2 82
AX2 10
RH2 68
OE2 146
MH1 7
AM1 6
AU4 2
KG1 59
AU3 6
AT7 4
KD1 7
AI6 6
AD1 7
AH1 6
KC1 7
AI2 6
AT1 9
GD4 19
AX3 5
AH4 6
KB3 7
AT2 13
AB1 7
AH5 6
ORIGIN 2
DEP_HOUR 24
FL_DATE 91

In [116]:
#Delete redundant columns
cols_to_delete1 = ['AB1','AD1','AE1','AK1','AH1','AH2','AH3','AH4','AH5','AH6','AI1','AI2','AI3','AI4',"AI5",'AI6','AM1','AU1','AU2','AU3','AU4',"AW1",'AW2','AW3','AW4','AW5','KC1','KC2','KD1',
                   'KD2','KE1','KG1','KG2','GA1','GA2','GA3','GE1','GF1','MH1','MG1','MK1','MW1','OD1',]
for col in cols_to_delete1:
  del df_weather[col]

In [117]:
for col in df_weather.columns:
  print(col, df_weather[col].nunique())

STATION 2
DATE 5849
SOURCE 2
LATITUDE 2
LONGITUDE 2
ELEVATION 2
NAME 2
REPORT_TYPE 4
CALL_SIGN 2
WND 636
CIG 98
VIS 28
TMP 122
DEW 116
SLP 384
KA1 194
KA2 185
MA1 213
MD1 237
OC1 26
REM 5863
EQD 170
AX4 2
GD1 343
AJ1 37
KA4 40
AN1 30
RH1 45
RH3 58
AX1 5
OE1 165
KA3 45
AA3 31
KB2 7
AT5 5
GD2 221
MF1 164
AA1 103
OE3 106
AT6 5
WA1 5
GD3 120
AT3 13
AT4 7
ED1 3
KB1 7
AA2 82
AX2 10
RH2 68
OE2 146
AT7 4
AT1 9
GD4 19
AX3 5
KB3 7
AT2 13
ORIGIN 2
DEP_HOUR 24
FL_DATE 91

In [118]:
#df_weather_copy = df_weather.copy()
#print(df_weather_copy.AA1.value_counts())
#print(df_weather_copy.AA2.value_counts())
#Index_label = df_weather_copy[df_weather_copy['AA2']!=""].index.tolist()
#print(Index_label)
#print(df_weather_copy['AA1'].iloc[65])
#print(df_weather_copy['AA2'].iloc[65])

In [119]:
#funciton to combine colomns, eg AA1, AA2, AA3 etc.
def combine_columns(df, col1, col2):
  mask = df[col2] != ""
  df.loc[mask, col1] = df[col2]
  del df[col2]
  return df

In [120]:
df_weather = combine_columns(df_weather, 'AA1','AA2')
df_weather = combine_columns(df_weather, 'AA1','AA3')
df_weather = combine_columns(df_weather, 'AT1','AT2')
df_weather = combine_columns(df_weather, 'AT1','AT3')
df_weather = combine_columns(df_weather, 'AT1','AT4')
df_weather = combine_columns(df_weather, 'AT1','AT5')
df_weather = combine_columns(df_weather, 'AT1','AT6')
df_weather = combine_columns(df_weather, 'AT1','AT7')
df_weather = combine_columns(df_weather, 'AX1','AX2')
df_weather = combine_columns(df_weather, 'AX1','AX3')
df_weather = combine_columns(df_weather, 'AX1','AX4')
df_weather = combine_columns(df_weather, 'GD1','GD2')
df_weather = combine_columns(df_weather, 'GD1','GD3')
df_weather = combine_columns(df_weather, 'GD1','GD4')
df_weather = combine_columns(df_weather, 'KA1','KA2')
df_weather = combine_columns(df_weather, 'KA1','KA3')
df_weather = combine_columns(df_weather, 'KA1','KA4')
df_weather = combine_columns(df_weather, 'KB1','KB2')
df_weather = combine_columns(df_weather, 'KB1','KB3')
df_weather = combine_columns(df_weather, 'OE1','OE2')
df_weather = combine_columns(df_weather, 'OE1','OE3')
df_weather = combine_columns(df_weather, 'RH1','RH2')
df_weather = combine_columns(df_weather, 'RH1','RH3')


In [121]:
for col in df_weather.columns:
  print(col, df_weather[col].nunique())

STATION 2
DATE 5849
SOURCE 2
LATITUDE 2
LONGITUDE 2
ELEVATION 2
NAME 2
REPORT_TYPE 4
CALL_SIGN 2
WND 636
CIG 98
VIS 28
TMP 122
DEW 116
SLP 384
KA1 225
MA1 213
MD1 237
OC1 26
REM 5863
EQD 170
GD1 312
AJ1 37
AN1 30
RH1 58
AX1 12
OE1 106
MF1 164
AA1 188
WA1 5
ED1 3
KB1 7
AT1 11
ORIGIN 2
DEP_HOUR 24
FL_DATE 91

In [122]:
#pull out the correct feature of the variable using index
cols_to_condense0 = ['AJ1','AX1','GD1','MF1','OC1','CIG','VIS']
cols_to_condense1 = ['AA1','AN1','AT1','WA1']
cols_to_condense2 = ['ED1','KA1','KB1','MA1','MD1','OE1','RH1']
cols_to_condense3 = ['WND']

def condense_col(df, col_name,num):
  df[col_name] = df[col_name].str.split(',').str[num]
  return df

for x in cols_to_condense0:
  df_weather = condense_col(df_weather, x, 0)
for y in cols_to_condense1:
  df_weather = condense_col(df_weather, y, 1)
for z in cols_to_condense2:
  df_weather = condense_col(df_weather, z, 2)
for a in cols_to_condense3:
  df_weather = condense_col(df_weather, a, 3)
df_weather.head()

STATION 
 DATE 
 SOURCE 
 LATITUDE 
 LONGITUDE 
 ELEVATION 
 NAME 
 REPORT_TYPE 
 CALL_SIGN 
 WND 
 CIG 
 VIS 
 TMP 
 DEW 
 SLP 
 KA1 
 MA1 
 MD1 
 OC1 
 REM 
 EQD 
 GD1 
 AJ1 
 AN1 
 RH1 
 AX1 
 OE1 
 MF1 
 AA1 
 WA1 
 ED1 
 KB1 
 AT1 
 ORIGIN 
 DEP_HOUR 
 FL_DATE 
 
 
 
 
 0 
 72530094846 
 2015-01-01 00:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 0057 
 22000 
 016093 
 -0078,5 
 -0172,5 
 10234,5 
 NaN 
 09974 
 NaN 
 0118 
 MET10412/31/14 18:51:03 METAR KORD 010051Z 240... 
 D01 0ADE726 
 1 
 
 NaN 
 NaN 
 
 NaN 
 
 0000 
 NaN 
 NaN 
 NaN 
 NaN 
 ORD 
 2 
 2015-01-01 
 
 
 1 
 72530094846 
 2015-01-01 01:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 0077 
 22000 
 016093 
 -0078,5 
 -0167,5 
 10235,5 
 NaN 
 09974 
 NaN 
 0113 
 MET12912/31/14 19:51:03 METAR KORD 010151Z 230... 
 D01 0ADE726 
 1 
 
 NaN 
 NaN 
 
 NaN 
 
 0000 
 NaN 
 NaN 
 NaN 
 NaN 
 ORD 
 3 
 2015-01-01 
 
 
 2 
 72530094846 
 2015-01-01 02:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 0072 
 22000 
 016093 
 -0078,5 
 -0172,5 
 10232,5 
 NaN 
 09974 
 009 
 0139 
 MET12812/31/14 20:51:03 METAR KORD 010251Z 240... 
 D01 0ADE726 
 1 
 
 NaN 
 NaN 
 
 NaN 
 
 0000 
 NaN 
 NaN 
 NaN 
 NaN 
 ORD 
 4 
 2015-01-01 
 
 
 3 
 72530094846 
 2015-01-01 03:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 0067 
 22000 
 016093 
 -0078,5 
 -0172,5 
 10229,5 
 NaN 
 09971 
 NaN 
 0103 
 MET11912/31/14 21:51:03 METAR KORD 010351Z 240... 
 
 0 
 
 NaN 
 NaN 
 
 NaN 
 
 0000 
 NaN 
 NaN 
 NaN 
 NaN 
 ORD 
 5 
 2015-01-01 
 
 
 4 
 72530094846 
 2015-01-01 04:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 0088 
 22000 
 016093 
 -0078,5 
 -0172,5 
 10223,5 
 NaN 
 09964 
 NaN 
 0124 
 MET10112/31/14 22:51:03 METAR KORD 010451Z 240... 
 
 0 
 
 NaN 
 NaN 
 
 NaN 
 
 0000 
 NaN 
 NaN 
 NaN 
 NaN 
 ORD 
 6 
 2015-01-01

In [123]:
#get rid of nans
df_weather = df_weather.fillna(0)

In [124]:
Y_continuous = df_weather[['AA1','AJ1','ED1','KA1','KB1','MA1','MD1','MF1','OC1','OE1','RH1','WA1']]
Y_continuous.head()

AA1 
 AJ1 
 ED1 
 KA1 
 KB1 
 MA1 
 MD1 
 MF1 
 OC1 
 OE1 
 RH1 
 WA1 
 
 
 
 
 0 
 0000 
 
 0 
 0 
 0 
 09974 
 0 
 
 0118 
 0 
 0 
 0 
 
 
 1 
 0000 
 
 0 
 0 
 0 
 09974 
 0 
 
 0113 
 0 
 0 
 0 
 
 
 2 
 0000 
 
 0 
 0 
 0 
 09974 
 009 
 
 0139 
 0 
 0 
 0 
 
 
 3 
 0000 
 
 0 
 0 
 0 
 09971 
 0 
 
 0103 
 0 
 0 
 0 
 
 
 4 
 0000 
 
 0 
 0 
 0 
 09964 
 0 
 
 0124 
 0 
 0 
 0

In [125]:
Y_continuous["AA1"] = pd.to_numeric(Y_continuous["AA1"], downcast="float")
Y_continuous["AJ1"] = pd.to_numeric(Y_continuous["AJ1"], downcast="float")
Y_continuous["ED1"] = pd.to_numeric(Y_continuous["ED1"], downcast="float")
Y_continuous["KA1"] = pd.to_numeric(Y_continuous["KA1"], downcast="float")
Y_continuous["KB1"] = pd.to_numeric(Y_continuous["KB1"], downcast="float")
Y_continuous["MA1"] = pd.to_numeric(Y_continuous["MA1"], downcast="float")
Y_continuous["MD1"] = pd.to_numeric(Y_continuous["MD1"], downcast="float")
Y_continuous["MF1"] = pd.to_numeric(Y_continuous["MF1"], downcast="float")
Y_continuous["OC1"] = pd.to_numeric(Y_continuous["OC1"], downcast="float")
Y_continuous["OE1"] = pd.to_numeric(Y_continuous["OE1"], downcast="float")
Y_continuous["RH1"] = pd.to_numeric(Y_continuous["RH1"], downcast="float")
Y_continuous["WA1"] = pd.to_numeric(Y_continuous["WA1"], downcast="float")

/local_disk0/tmp/1595169103376-0/PythonShell.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 from __future__ import absolute_import
/local_disk0/tmp/1595169103376-0/PythonShell.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 from __future__ import print_function
/local_disk0/tmp/1595169103376-0/PythonShell.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 
/local_disk0/tmp/1595169103376-0/PythonShell.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import errno
/local_disk0/tmp/1595169103376-0/PythonShell.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import linecache
/local_disk0/tmp/1595169103376-0/PythonShell.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import os
/local_disk0/tmp/1595169103376-0/PythonShell.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import pickle
/local_disk0/tmp/1595169103376-0/PythonShell.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import resource
/local_disk0/tmp/1595169103376-0/PythonShell.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import signal
/local_disk0/tmp/1595169103376-0/PythonShell.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import six
/local_disk0/tmp/1595169103376-0/PythonShell.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
 import six.moves.queue
/local_disk0/tmp/1595169103376-0/PythonShell.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

In [126]:
Y_continuous.head()

AA1 
 AJ1 
 ED1 
 KA1 
 KB1 
 MA1 
 MD1 
 MF1 
 OC1 
 OE1 
 RH1 
 WA1 
 
 
 
 
 0 
 0.0 
 NaN 
 0.0 
 0.0 
 0.0 
 9974.0 
 0.0 
 NaN 
 118.0 
 0.0 
 0.0 
 0.0 
 
 
 1 
 0.0 
 NaN 
 0.0 
 0.0 
 0.0 
 9974.0 
 0.0 
 NaN 
 113.0 
 0.0 
 0.0 
 0.0 
 
 
 2 
 0.0 
 NaN 
 0.0 
 0.0 
 0.0 
 9974.0 
 9.0 
 NaN 
 139.0 
 0.0 
 0.0 
 0.0 
 
 
 3 
 0.0 
 NaN 
 0.0 
 0.0 
 0.0 
 9971.0 
 0.0 
 NaN 
 103.0 
 0.0 
 0.0 
 0.0 
 
 
 4 
 0.0 
 NaN 
 0.0 
 0.0 
 0.0 
 9964.0 
 0.0 
 NaN 
 124.0 
 0.0 
 0.0 
 0.0

In [127]:
Y_continuous = Y_continuous.fillna(0)

In [128]:
Y = StandardScaler().fit_transform(Y_continuous)
Y

/databricks/python/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
 return self.partial_fit(X, y)
/databricks/python/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype float32 were all converted to float64 by StandardScaler.
 return self.fit(X, **fit_params).transform(X)
Out[88]: array([[-0.17130727, -0.191526 , -0.2770698 , ..., -0.16571591,
 -0.17221165, -0.081162 ],
 [-0.17130727, -0.191526 , -0.2770698 , ..., -0.16571591,
 -0.17221165, -0.081162 ],
 [-0.17130727, -0.191526 , -0.2770698 , ..., -0.16571591,
 -0.17221165, -0.081162 ],
 ...,
 [-0.17130727, -0.191526 , -0.2770698 , ..., -0.16571591,
 -0.17221165, -0.081162 ],
 [-0.17130727, -0.191526 , -0.2770698 , ..., -0.16571591,
 -0.17221165, -0.081162 ],
 [-0.17130727, -0.191526 , -0.2770698 , ..., -0.16571591,
 -0.17221165, -0.081162 ]])

In [129]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(Y)
df_principal1 = pd.DataFrame(data = principalComponents
             , columns = ['weather principal component 1', 'weather principal component 2']).fillna(0)
df_principal1.head()

weather principal component 1 
 weather principal component 2 
 
 
 
 
 0 
 -0.500590 
 -0.016299 
 
 
 1 
 -0.495111 
 -0.026880 
 
 
 2 
 -0.563419 
 0.513367 
 
 
 3 
 -0.483340 
 -0.048177 
 
 
 4 
 -0.504452 
 -0.004056

In [130]:
# get non-continuous variables
df_weather_non_continuous = df_weather.drop(['AA1','AJ1','ED1','KA1','KB1','MA1','MD1','MF1','OC1','OE1','RH1','WA1'], axis=1)
df_weather_non_continuous.shape

Out[90]: (5863, 24)

In [131]:
# concat weather PCA continuous and non-continuous variables
df_weather_pca = pd.concat([df_weather_non_continuous, df_principal1], axis=1)
df_weather_pca.shape

Out[91]: (5863, 26)

### 7. Joining the Datasets
Join the airline and weather dataset by location and time
- Location:
- Time: 
The flight data needs to be joined to the weather data, but the weather data must be two hours ahead of the flight departure

#### 7-1. Check airline data's location and time
- Location: ORIGIN
- Time: 
  - ADJ_DEP_TIME -> DEP_HOUR
  - FL_DATE

In [134]:
# When the row do not have departure time in ADJ_DEP_TIME than drop it
df_airlines_pca = df_airlines_pca.drop(df_airlines_pca[df_airlines_pca.ADJ_DEP_TIME.isna()].index)

In [135]:
# Extract the departure hour and date from ADJ_DEP_TIME column
df_airlines_pca['DEP_HOUR'] = df_airlines_pca.ADJ_DEP_TIME.map(lambda x: str(x)[0:-4])
df_airlines_pca.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 FLIGHTS 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DELAY_15 
 ADJ_DEP_TIME 
 principal component 1 
 principal component 2 
 DEP_HOUR 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 1040.0 
 1.502625 
 0.007499 
 10 
 
 
 1 
 2015-01-02 
 AA 
 19805 
 AA 
 N3GFAA 
 1375 
 ORD 
 RSW 
 1035 
 1051.0 
 9.0 
 1100.0 
 1432.0 
 5.0 
 1415 
 1437.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1051.0 
 1.575095 
 0.268880 
 10 
 
 
 2 
 2015-01-03 
 AA 
 19805 
 AA 
 N3LUAA 
 1375 
 ORD 
 RSW 
 1035 
 1041.0 
 22.0 
 1103.0 
 1437.0 
 20.0 
 1415 
 1457.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 1041.0 
 1.835153 
 -0.554518 
 10 
 
 
 3 
 2015-01-04 
 AA 
 19805 
 AA 
 N3HXAA 
 1375 
 ORD 
 RSW 
 1035 
 1107.0 
 23.0 
 1130.0 
 1519.0 
 4.0 
 1415 
 1523.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1107.0 
 1.966342 
 -1.303872 
 11 
 
 
 4 
 2015-01-05 
 AA 
 19805 
 AA 
 N3CXAA 
 1375 
 ORD 
 RSW 
 1035 
 1058.0 
 21.0 
 1119.0 
 1444.0 
 5.0 
 1415 
 1449.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1058.0 
 1.559364 
 0.293698 
 10

#### 7-2. Join the airline and weather data by location and time
- Location:
  - ORIGIN
- Time:
  - FL_DATE
  - DEP_HOUR

##### Check FL_DATE

In [138]:
df_airlines_pca['FL_DATE'] = pd.to_datetime(df_airlines_pca.FL_DATE).apply(lambda x: x.date())
print(df_airlines_pca.FL_DATE[0], type(df_airlines_pca.FL_DATE[0]))
df_airlines_pca.FL_DATE.value_counts()

2015-01-01 <class 'datetime.date'>
Out[94]: 2015-03-30 1986
2015-03-27 1975
2015-03-16 1973
2015-03-26 1972
2015-03-13 1971
2015-03-19 1967
2015-03-12 1966
2015-03-09 1962
2015-03-31 1957
2015-03-25 1955
2015-03-18 1953
2015-03-20 1949
2015-03-11 1944
2015-03-17 1938
2015-03-24 1937
2015-03-06 1930
2015-03-10 1918
2015-02-13 1905
2015-03-02 1899
2015-03-29 1897
2015-02-19 1893
2015-03-15 1889
2015-02-27 1881
2015-03-22 1880
2015-02-12 1868
2015-03-08 1866
2015-02-20 1861
2015-02-23 1858
2015-02-18 1856
2015-02-17 1845
 ... 
2015-01-04 1707
2015-01-05 1699
2015-01-06 1696
2015-01-26 1688
2015-03-01 1687
2015-01-25 1640
2015-01-03 1639
2015-02-08 1633
2015-03-14 1619
2015-03-21 1617
2015-02-03 1615
2015-03-28 1612
2015-03-07 1608
2015-01-11 1596
2015-01-08 1591
2015-02-25 1588
2015-01-27 1560
2015-02-15 1530
2015-01-18 1526
2015-01-01 1422
2015-02-28 1409
2015-02-02 1375
2015-02-21 1372
2015-01-24 1329
2015-02-07 1328
2015-01-10 1293
2015-01-17 1286
2015-02-14 1284
2015-01-31 1263
2015-02-01 878
Name: FL_DATE, Length: 90, dtype: int64

In [139]:
print(df_weather_pca.FL_DATE[0], type(df_weather_pca.FL_DATE[0]))
df_weather_pca.FL_DATE.value_counts()

2015-01-01 <class 'datetime.date'>
Out[95]: 2015-02-26 132
2015-01-04 124
2015-03-03 106
2015-03-23 101
2015-01-12 100
2015-01-09 90
2015-01-03 90
2015-02-09 84
2015-02-02 83
2015-02-04 83
2015-02-01 80
2015-02-24 77
2015-03-27 76
2015-01-26 75
2015-03-20 74
2015-01-27 74
2015-01-24 73
2015-03-05 73
2015-03-25 73
2015-03-01 73
2015-02-17 72
2015-03-26 72
2015-02-14 71
2015-03-04 71
2015-01-15 70
2015-01-25 70
2015-01-06 70
2015-01-13 70
2015-03-10 69
2015-02-22 68
 ... 
2015-02-03 56
2015-01-30 56
2015-01-20 55
2015-02-18 55
2015-03-29 55
2015-01-11 55
2015-02-19 54
2015-03-15 53
2015-03-24 53
2015-01-28 52
2015-02-27 52
2015-02-12 52
2015-02-13 51
2015-03-07 51
2015-01-10 50
2015-02-07 50
2015-03-09 50
2015-03-28 50
2015-03-08 50
2015-01-19 50
2015-03-18 50
2015-03-16 50
2015-01-17 50
2015-01-31 50
2015-02-05 50
2015-02-06 50
2015-01-07 50
2015-02-28 48
2015-01-01 48
2015-04-01 4
Name: FL_DATE, Length: 91, dtype: int64

##### Check ORIGIN

In [141]:
print(df_airlines_pca.reset_index().ORIGIN[0], type(df_airlines_pca.reset_index().ORIGIN[0]))
df_airlines_pca.ORIGIN.value_counts()

ORD <class 'str'>
Out[96]: ATL 88168
ORD 67905
Name: ORIGIN, dtype: int64

In [142]:
print(df_weather_pca.ORIGIN[1], type(df_weather_pca.ORIGIN[1]))
df_weather_pca.ORIGIN.value_counts()

ORD <class 'str'>
Out[97]: ORD 3010
ATL 2853
Name: ORIGIN, dtype: int64

##### Check DEP_HOUR

In [144]:
print(df_airlines_pca.DEP_HOUR[0], type(df_airlines_pca.DEP_HOUR[0]))
df_airlines_pca.DEP_HOUR.value_counts()

10 <class 'str'>
Out[98]: 9 11426
10 11316
13 11261
8 11201
19 11145
15 10851
20 10782
16 10596
17 9309
12 9019
21 8799
14 8253
18 8127
11 7870
7 5997
22 4415
6 3114
5 1428
23 836
 143
4 119
1 50
2 9
24 6
3 1
Name: DEP_HOUR, dtype: int64

In [145]:
df_weather_pca['DEP_HOUR'] = df_weather_pca.DEP_HOUR.astype(str)
print(df_weather_pca.DEP_HOUR[0], type(df_weather_pca.DEP_HOUR[0]))
df_weather_pca.DEP_HOUR.value_counts()

2 <class 'str'>
Out[99]: 6 319
7 318
16 263
13 258
17 257
12 250
22 246
11 245
21 245
14 244
15 240
20 239
18 239
8 238
10 236
1 235
19 234
9 232
23 231
0 228
4 220
2 219
5 218
3 209
Name: DEP_HOUR, dtype: int64

In [146]:
df_weather_unique = df_weather_pca.drop_duplicates(['DEP_HOUR','FL_DATE','ORIGIN'])
#df_weather.head()

In [147]:
print(len(df_weather))
print(len(df_weather_unique))

5863
4320

##### Merge airline and weather data

In [149]:
# left join of airline and weather data by location and time
df = pd.merge(df_airlines_pca, df_weather_unique, how='inner', on=['ORIGIN', 'FL_DATE', 'DEP_HOUR'])
print(df.shape)
df.head()

FL_DATE 
 OP_UNIQUE_CARRIER 
 OP_CARRIER_AIRLINE_ID 
 OP_CARRIER 
 TAIL_NUM 
 OP_CARRIER_FL_NUM 
 ORIGIN 
 DEST 
 CRS_DEP_TIME 
 DEP_TIME 
 TAXI_OUT 
 WHEELS_OFF 
 WHEELS_ON 
 TAXI_IN 
 CRS_ARR_TIME 
 ARR_TIME 
 CANCELLED 
 CANCELLATION_CODE 
 DIVERTED 
 FLIGHTS 
 FIRST_DEP_TIME 
 TOTAL_ADD_GTIME 
 LONGEST_ADD_GTIME 
 DIV_AIRPORT_LANDINGS 
 DIV_REACHED_DEST 
 DIV1_AIRPORT 
 DIV1_AIRPORT_ID 
 DIV1_AIRPORT_SEQ_ID 
 DIV1_WHEELS_ON 
 DIV1_TOTAL_GTIME 
 DIV1_LONGEST_GTIME 
 DIV1_WHEELS_OFF 
 DIV1_TAIL_NUM 
 DIV2_AIRPORT 
 DIV2_AIRPORT_ID 
 DIV2_AIRPORT_SEQ_ID 
 DIV2_WHEELS_ON 
 DIV2_TOTAL_GTIME 
 DIV2_LONGEST_GTIME 
 DIV2_WHEELS_OFF 
 DIV2_TAIL_NUM 
 DIV3_AIRPORT 
 DIV3_AIRPORT_ID 
 DIV3_AIRPORT_SEQ_ID 
 DIV3_WHEELS_ON 
 DIV3_TOTAL_GTIME 
 DIV3_LONGEST_GTIME 
 DIV3_WHEELS_OFF 
 DIV3_TAIL_NUM 
 DELAY_15 
 ADJ_DEP_TIME 
 principal component 1 
 principal component 2 
 DEP_HOUR 
 STATION 
 DATE 
 SOURCE 
 LATITUDE 
 LONGITUDE 
 ELEVATION 
 NAME 
 REPORT_TYPE 
 CALL_SIGN 
 WND 
 CIG 
 VIS 
 TMP 
 DEW 
 SLP 
 REM 
 EQD 
 GD1 
 AN1 
 AX1 
 AT1 
 weather principal component 1 
 weather principal component 2 
 
 
 
 
 0 
 2015-01-01 
 AA 
 19805 
 AA 
 N3AKAA 
 1375 
 ORD 
 RSW 
 1035 
 1040.0 
 15.0 
 1055.0 
 1419.0 
 6.0 
 1415 
 1425.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 1040.0 
 1.502625 
 0.007499 
 10 
 72530094846 
 2015-01-01 08:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 0077 
 22000 
 016093 
 -0089,5 
 -0156,5 
 10197,5 
 MET12501/01/15 02:51:02 METAR KORD 010851Z 250... 
 
 0 
 0 
 
 0 
 -0.574722 
 0.824638 
 
 
 1 
 2015-01-01 
 B6 
 20409 
 B6 
 N267JB 
 906 
 ORD 
 JFK 
 956 
 1009.0 
 13.0 
 1022.0 
 1252.0 
 7.0 
 1259 
 1259.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 1009.0 
 -0.053968 
 0.015656 
 10 
 72530094846 
 2015-01-01 08:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 0077 
 22000 
 016093 
 -0089,5 
 -0156,5 
 10197,5 
 MET12501/01/15 02:51:02 METAR KORD 010851Z 250... 
 
 0 
 0 
 
 0 
 -0.574722 
 0.824638 
 
 
 2 
 2015-01-01 
 EV 
 20366 
 EV 
 N17560 
 6159 
 ORD 
 MCI 
 1025 
 1019.0 
 14.0 
 1033.0 
 1145.0 
 4.0 
 1203 
 1149.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 0.0 
 1019.0 
 -0.908224 
 -0.001567 
 10 
 72530094846 
 2015-01-01 08:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 0077 
 22000 
 016093 
 -0089,5 
 -0156,5 
 10197,5 
 MET12501/01/15 02:51:02 METAR KORD 010851Z 250... 
 
 0 
 0 
 
 0 
 -0.574722 
 0.824638 
 
 
 3 
 2015-01-01 
 MQ 
 20398 
 MQ 
 N508MQ 
 3094 
 ORD 
 ELP 
 1025 
 1052.0 
 10.0 
 1102.0 
 1324.0 
 3.0 
 1243 
 1327.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 1.0 
 1052.0 
 2.783813 
 0.236747 
 10 
 72530094846 
 2015-01-01 08:51:00 
 7 
 41.995 
 -87.9336 
 201.8 
 CHICAGO OHARE INTERNATIONAL AIRPORT, IL US 
 FM-15 
 KORD 
 0077 
 22000 
 016093 
 -0089,5 
 -0156,5 
 10197,5 
 MET12501/01/15 02:51:02 METAR KORD 010851Z 250... 
 
 0 
 0 
 
 0 
 -0.574722 
 0.824638 
 
 
 4 
 2015-01-01 
 MQ 
 20398 
 MQ 
 N694MQ 
 3180 
 ORD 
 BIS 
 1050 
 1043.0 
 18.0 
 1101.0 
 1303.0 
 2.0 
 1303 
 1305.0 
 0.0 
 None 
 0.0 
 1.0 
 NaN 
 NaN 
 NaN 
 0 
 NaN 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 None 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 NaN 
 None 
 N

####Phase II summary: Scalability, Efficiency, Distributed/parallel Training and Scoring Pipeline
1. Create a new outcome variables "DELAY_15" to check if the departure time is delay over 15 mins.
2. Remove redundant variables and variables related to the outcome.
3. Create a new outcome variables "ADJ_DEP_TIME" as the real departure time.
4. Standardize the continuous variables.
5. Dimensionality reduction use PCA.
6. Join the airlines and weather data by location and time.

# Algorithm Exploration
Apply 2 to 3 algorithms to the training set, and discuss expectations, trade-offs, and results. Do not over fine tune these. You will want to use this process to select a final algorithm which you will spend your efforts on fine tuning.

### Splitting the Data

We're splitting the data into 2 sets. A train_test set that will be used with k-fold cross-validation contains 80% of the data. Lastly, a hold out set of 20% of the data to be used as a final check of the algorithm once we have one that is performant.

In [153]:
## Training, Test, Holdout Splits
train_test_df, holdout_df = train_test_split(df, test_size=0.2, random_state = 42)
train_df, test_df = train_test_split(train_test_df, test_size=0.25, random_state = 42)

In [154]:
x_vars = ['CANCELLED','principal component 1','principal component 2', 'weather principal component 1','weather principal component 2']
X = train_test_df[x_vars]
y = train_test_df['DELAY_15']
X_test = test_df[x_vars]
y_test = test_df['DELAY_15']
X_train = train_df[x_vars]
y_train = train_df['DELAY_15']
random_state = 42
leaf_n_dict = {}
knn_n_dict = {}
logreg_setup_ls = [('liblinear','l1'),('liblinear','l2')]

In [155]:
#def logreg_modeler(C, X=X,y=y,X_test=X_test,y_test=y_test):
#  model = LogisticRegression(solver='lbfgs', C=C, max_iter=1000).fit(X,y)
#  model1_pred = model1_pred = model1.predict(X_test)
#  score = np.round(accuracy_score(y_test,model1_pred),4)
#  print("Model 1 accuracy: ", score)
#  logreg_C_dict[C] = score
  

In [156]:
#model1 = LogisticRegression(random_state=42, solver='lbfgs').fit(X,y)
#model1_pred = model1.predict(X_test)

#print("Model 1 accuracy: ", np.round(accuracy_score(y_test,model1_pred),3))
#confusion_matrix(y_test,model1_pred)

In [157]:
for solver,penalty in logreg_setup_ls:
  scores = cross_validate(LogisticRegression(random_state=42, solver=solver, penalty=penalty),X,y,cv=5)
  print(f"Accuracy scores for {penalty} Logistic Regression: ",scores['test_score'])

Accuracy scores for l1 Logistic Regression: [0.76656379 0.76695527 0.76631393 0.76686576 0.7667455 ]
Accuracy scores for l2 Logistic Regression: [0.76656379 0.76695527 0.76631393 0.76686576 0.7667455 ]

In [158]:
#model2 = tree.DecisionTreeClassifier(random_state=42, max_leaf_nodes=20).fit(X,y)
#model2_pred = model2.predict(X_test)

#print("Model 2 accuracy: ", np.round(accuracy_score(y_test,model2_pred),3))
#confusion_matrix(y_test,model2_pred)

In [159]:
#model3 = RandomForestClassifier(random_state=42, max_leaf_nodes=20).fit(X,y)
#model3_pred = model3.predict(X_test)

#print("Model 3 accuracy: ", np.round(accuracy_score(y_test,model3_pred),3))
#confusion_matrix(y_test,model3_pred)

In [160]:
def randomforestmodeler(random_state, leaf_n):
  model = RandomForestClassifier(random_state=random_state, max_leaf_nodes=leaf_n, n_estimators=10)
  scores = cross_validate(model, X,y,cv=5)
  #model_pred = model.predict(X_test)
  #score = np.round(accuracy_score(y_test,model_pred),3)
  #print(f"Model {leaf_n} accuracy: ", score)
  leaf_n_dict[leaf_n] = scores['test_score']

In [161]:
leafs_ls = [x*5 for x in range(1,40)]
for leaf_n in leafs_ls:
  randomforestmodeler(random_state, leaf_n)
pd.DataFrame.from_dict(leaf_n_dict, orient='index').plot(title='K-fold Random Forest: Accuracy vs. Max Leaf Nodes')

In [162]:
#model4 = KNeighborsClassifier(n_neighbors=5).fit(X,y)
#model4_pred = model4.predict(X_test)

#print("Model 4 accuracy: ", np.round(accuracy_score(y_test,model4_pred),3))
#confusion_matrix(y_test,model4_pred)

In [163]:
def knnmodeler(n):
  model = KNeighborsClassifier(n_neighbors=n)
  scores = cross_validate(model,X,y,cv=5)
  #model_pred = model.predict(X_test)
  #score = np.round(accuracy_score(y_test,model_pred),3)
  knn_n_dict[n] = scores['test_score']

In [164]:
n_ls = [x for x in range(1,10)]
for n in n_ls:
  knnmodeler(n)
pd.DataFrame.from_dict(knn_n_dict, orient='index').plot(title='K-fold KNN: Accuracy vs. N_Neighbors')

In [165]:
def stack_model(X_train,y_train, X_test):
  knn_model = KNeighborsClassifier(n_neighbors=8).fit(X_train,y_train)
  knn_model_pred_train = knn_model.predict(X_train)
  knn_model_pred_test = knn_model.predict(X_test)
  
  randfor_model = RandomForestClassifier(random_state=random_state, max_leaf_nodes=100, n_estimators=10).fit(X_train,y_train)
  randfor_pred_train = randfor_model.predict(X_train)
  randfor_pred_test = randfor_model.predict(X_test)

  logreg_l2_model = LogisticRegression(random_state=random_state, solver='liblinear',penalty='l2').fit(X_train,y_train)
  logreg_l2_pred_train = logreg_l2_model.predict(X_train)
  logreg_l2_pred_test = logreg_l2_model.predict(X_test)
  
  return list(zip(logreg_l2_pred_train, randfor_pred_train, knn_model_pred_train)),\
         list(zip(logreg_l2_pred_test, randfor_pred_test, knn_model_pred_test))

In [166]:
stacked_x_vars_train, stacked_x_vars_test = stack_model(X_train, y_train, X_test)
stacked_model = tree.DecisionTreeClassifier(random_state=random_state).fit(stacked_x_vars_train,y_train)

stacked_model_pred = stacked_model.predict(stacked_x_vars_test)
accuracy_score(y_test,stacked_model_pred)

Out[116]: 0.9037037037037037

# Algorithm Implementation 
Create your own toy example that matches the dataset provided and use this toy example to explain the math behind the algorithm that you will perform. Apply your algorithm to the training dataset and evaluate your results on the test set.

# Conclusions
Report results and learnings for both the ML as well as the scalability.

# Application of Course Concepts 
Pick 3-5 key course concepts and discuss how your work on this assignment illustrates an understanding of these concepts.